## React
- ReactJS is an open-source JavaScript library that is used for building user interfaces in a declarative and efficient way.

- It is a component-based front-end library responsible only for the view layer of an MVC (Model View Controller) architecture. React is used to create modular user interfaces and it promotes the development of reusable UI components that display dynamic data.

- Note: React is not a framework. It is just a library developed by Facebook to solve some problems that we were facing earlier.

### JSX

#### Animal Fun Facts
In this project, we’ll build a program that allows users to click an animal on the screen in order to have a fun fact pop up.

A preview that displays the final result of the Animal Fun Facts project. A user clicks on images of different animals, which displays a fun fact about them.

Our program will display a selection of animals on the screen. We’ll be allowed to decide if we want to include a background or not. Clicking an animal will cause a fact to be randomly selected from a list of potential options. The selected fact will pop up on the screen. As we keep clicking, we’ll be able to see different facts.

If you get stuck during this project or would like to see an experienced developer work through it, click “Get Unstuck“ to see a project walkthrough video.

Let’s get started!

**Tasks**
**Add a Title**
1. On line 1 in app.js, you should see an import statement. This is importing the animals object from animals.js. Feel free to take a look at it now, but we’ll be coming back to it in later tasks.

For now, let’s import the React from react and createRoot from react-dom/client.


2. Click on index.html to find out the id of the HTML element to get a reference of. Use this id and the document object to get a reference of this element and store it in a constant called container.


3. Create a React root so you can render content using the createRoot() method, passing container as an argument. Store this root in a constant called root.


4. Add a title constant that will hold the value of the title. For now, set its value to an empty string.

In addition, create an animalFacts constant to hold the JSX expression that we’ll want to be compiled. Set its value to a <h1> element that contains our title.

We still shouldn’t see anything in the browser yet! We’ll have to wait until we write our React root’s render() method before anything shows up.


5. We could fill in the empty string assigned to title if we wanted, but we could also leave it blank and let the JSX use a default value instead.

Using the ternary operator, let the <h1> heading use ‘Click an animal for a fun fact’ as the default if title is an empty string.


6. It’s time to call our root‘s render() method.

Let’s pass in animalFacts as the JSX expression that we want to be compiled and rendered.

When finished, click Save. If all goes well, we should see the text ‘Click an animal for a fun fact!’ appear on the screen!


**Add a Background**
7. Let’s add a background!

Somewhere above where animalFacts is defined, create a constant named background. Set its value to a <img /> element.

Now let’s give it some attributes:

Give it a class of 'background'
Let’s use 'ocean' for alt.
Finally, use '/images/ocean.jpg' as the value of src.


8. Let’s reformat the JSX expression stored in animalFacts to include the background variable.

Wrap the current <h1> element and our new background variable inside of <div></div> tags. Since the expression is going to be multiple lines, wrap it in parentheses.

Click Save. If everything is working as it should, we should see our background image showing up underneath the title!


**Add an Array of Images**
9. Use a for...in loop to iterate over the animals object that we’re importing on line 1. Before the animalFacts definition, define an images array. For each animal, add a new <img /> to that array.

Assuming animal is the placeholder variable in your for...in loop, each image should have the following attributes:

key: {animal}
className: 'animal'
alt: {animal}
src: {animals[animal].image}
aria-label: {animal}
role: 'button'


10. Now that we have our array of images, let’s inject it into the JSX expression.

Within the animalFacts JSX, underneath {background}, create a <div>. Give it a className attribute and set it equal to 'animals'. Nest the array of images inside of this element.

Finally, click Save. We should see our animals!


**Adding Fun Facts**
11. Now that we have our animals displaying on the screen, we’re ready to add an event listener! But first, let’s write a function to handle this event.

Create a function displayFact() that takes one parameter e, the event. We want this function to pick a random fun fact based on the selected animal.
Inside of the function, use e.target.alt to get the name of the animal being clicked.
Generate a random index and use it to access an element in the animal’s .facts array.
Save the fun fact in a variable.


12. We need a place to display our facts. Create an empty <p> element in animalFacts and give it an id attribute equal to 'fact'.


13. We’ll need to include the event listener with each <img> and edit the event listener so that it displays the fact in our new <p> element.

In the for...in loop, inside each <img>, add an onClick event listener that calls displayFact.
Inside displayFact() use document.getElementById('fact') to grab the <p> element where we’ll add our fact. Change the .innerHTML of the <p> element to our randomly selected fact.
Now save the code and click on an animal. We should see a fact pop up on the screen!


**Extra Credit**
14. Let’s add one last feature to our awesome app!

Create a showBackground constant. You can set its value to either true or false.

If showBackground is true, background should show up. If it’s false, it should not. Use the && operator in animalFacts to implement this feature.

Toggle the value of showBackground between true and false and save the code to see if you got it working!

Optional Task: In addition to the AND && operator, we can use the OR || operator. Given a list of variables or expressions, || will return the value of the first one whose boolean evaluates to true.

Considering that the boolean of an empty string is false, can you think of a way to use || to replace the ternary operator in the heading?

In [ ]:
// #+name: app.js
import { animals } from './animals';
import React from 'react';
import { createRoot } from 'react-dom/client';

const container = document.getElementById('app');
const root = createRoot(container);
const background = (
<img className='background' alt='ocean' src='/images/ocean.jpg' />
);

const showBackground = true;
const title = '';

const images = [];
// For each animal, add a new <img /> to that array.
for (const animal in animals) 
{
  images.push(<img key={animal} className='animal' alt={animal} src={animals[animal].image} aria-label='{animal}' role='button' onClick={displayFact}/>)
};

// function to pick a random fact based on selected animal
function displayFact(e) {

  // event listener targetting alt to get name of animal being clicked
  const animal = e.target.alt;

  //generate random index
  const randIndex = Math.floor(Math.random() * animals[animal].facts.length);
  console.log(randIndex);

  //save the fun fact in a variable
  const selectAnimalFact = animals[animal].facts[randIndex];

  document.getElementById('fact').innerHTML = selectAnimalFact;
}

const animalFacts = (
  <div>
    <h1>{title || 'Click an animal for a fun fact'}</h1>
    {showBackground && background}
    <div className='animals'>{images}</div>
    <p id='fact'></p>
  </div>
);

root.render(animalFacts);

In [ ]:
<!-- #+name: styles.css -->
div {
	display: flex;
	align-items: center;
	flex-direction: column;
}

.background {
	position: absolute;
	margin-top: 80px;
	width: 95%;
	z-index: 1
}

#fact {
	display: flex;
	justify-content: center;
  position: fixed;
	margin-top: 100px;
	width: 80%;
	font-size: 5vw;
	z-index: 2;
}

.animals {
	display: flex;
	flex-direction: row;
	position: fixed;
	width: 90%;
	margin-top: 240px;
	z-index: 2
}

.animal {
	width: 33.3%;
}

@media only screen and (max-width: 670px) {
	.animals {
		margin-top: 180px;
	}

  h1 {
    font-size: 24px;
  }
}

@media only screen and (max-width: 390px) {
	.animals {
		margin-top: 150px;
	}
}

@media only screen and (max-width: 300px) {
	.animals {
		margin-top: 130px;
	}
}

@media only screen and (max-width: 200px) {
	.animals {
		margin-top: 120px;
	}
}

### React Components

#### Authorization Form
A client just called you to say that they love their new website! There’s only one problem: they don’t like how their contact page displays their personal information for all to see.

They’ve asked you to hide their website’s contact page behind a password form. In this project, you’ll accomplish this by creating a React component to set up a simple authorization layer.

If you get stuck during this project or would like to see an experienced developer work through it, click “Get Unstuck“ to see a project walkthrough video.

Let’s get started!

Tasks
**Setting Up**
1. Click Save to see the current state of things.

The contact info in the browser looks fine, but it should be hidden until you enter a password!

Look in Contact.js. You can see a Contact function component. Inside is a function called handleSubmit(), which will be responsible for authorizing the user into the system.

There’s a lot of logic already here. As we progress, you’ll start learning what useState and such is. For now, just know that you can check whether a user has entered the right password by checking if authorized is true.

2. Let’s look at the <h1></h1> tags in the return statement.

Right now, the <h1> element displays the text Contact. If a user hasn’t been authorized, then you want the <h1> element to display Enter the Password instead.

Using what you know of conditionals in components, make the <h1> element display "Contact" only if authorized is true. If authorized is false, then the <h1> element should display "Enter the Password".


3. Let’s check to see if that last step is working properly.

For now, the browser should say, “Enter the Password”. This is because authorized has the initial value of false.

Edit line 5 so that it has useState(true) instead of useState(false) for now. You should see that the text now says “Contact”. Don’t worry about how useState works—we’ll go over it in a future React lesson.

If it works, then make sure to change it back to useState(false) before moving on.


**The Login Form**
4. If the user isn’t authorized, then you want them to see a login form into which they can enter a password. Let’s make that login form!

Before the return statement but after the handleSubmit() function, declare a new variable named login.

Set login equal to a JSX <form></form> element. This <form></form> is going to have multiple children, so wrap it in parentheses!

Give the <form></form> an attribute of action="#" to make sure it does not redirect.


5. Good! Now let’s give your form some <input />s for the user to fill out.

In between the <form></form> tags, write two <input /> tags. Give the first <input /> two attributes:

type="password"
placeholder="Password"
Give the second <input /> one attribute: type="submit".


**The Contact Info**
6. Now, let’s hide the contact info.

After your login variable, declare another variable named contactInfo. Set it equal to empty parentheses:

const contactInfo = (
); 
Make sure it is still inside the function component and before the return statement.

Next, move the <ul></ul> element in the return statement in between the parentheses we just created for contactInfo!


7. Great! By saving two JSX expressions as variables, you are ready to toggle between them.

In the component’s return statement, make a new line right below the <h1></h1> element. On this new line, use a ternary operator. If authorized is true, make the ternary return contactInfo. Otherwise, make the ternary return login.


8. Within the Contact function component, you see a function named handleSubmit().

This function will check whether a submitted password is equal to 'swordfish'. If it is, then authorized will become true.

You need handleSubmit to get called whenever a user clicks the submit button.

Give the <form></form> element an onSubmit attribute. Set the attribute’s value equal to the handleSubmit function.


9. Try entering an incorrect password and hitting “Submit”. Nothing should happen.

Now try entering “swordfish”. The website should reveal the contact info!

In [ ]:
// #+name: Contact.js
import React, { useState } from 'react';

function Contact() {
  const password = 'swordfish';
  const [authorized, setAuthorized] = useState(false);

  function handleSubmit(e) {
    const enteredPassword = e.target.querySelector(
      'input[type="password"]').value;
    const auth = enteredPassword == password;
    setAuthorized(auth)
  }

  // login form
  const login = (
    <form action="#" onSubmit={handleSubmit}>
      <input type="password" placeholder="Password" />
      <input type="submit" />
    </form>);

  // hiding the contact info
  const contactInfo = (
    <ul>
      <li>
        client@example.com
      </li>
      <li>
        555.555.5555
      </li>
    </ul>
  );
  
  return (
      <div id="authorization">
        <h1>{authorized ? "Contact" : "Enter the Password"}</h1>
        {authorized ? contactInfo : login}
      </div>
  );
}

export default Contact;

In [ ]:
<!-- #+name: style.css -->
html, body {
	margin: 0;
	height: 100%;
}

body {
	background-color: #ffffff;
	font-family: Helvetica, Arial, sans-serif;
  text-align: center;
}

#app {
	position: relative;
	height: 100%;
	width: 100%;
	padding-top: 10px;
}

#app div div {
	height: 100%;
}

#app div div div {
	position: relative;
	height: auto;
}

h1, h2 {
	margin-left: 5%;
	margin-right: 5%;
}

ul {
	list-style-type: none;
	padding: 0;
}

label {
	display: block;
	margin: 20px;
	font-size: 30px;
	font-weight: bold;
}

nav a {
	margin:12px;
	text-transform: uppercase;
	font-size: 10px;
}

button {
	-webkit-transition-duration: 0.1s; /* Safari */
	transition-duration: 0.1s;
	background-color: #F4595B;
	border-radius: 8px;
	border-bottom: 4px solid #C24648;
	color: white;
	padding: 15px 32px;
	text-align: center;
	text-decoration: none;
	display: inline-block;
	font-size: 16px;
	font-family: 'Oxygen', sans-serif;
	letter-spacing: 2px;
}

button:hover {
  background-color: #FF7375;
  border: none;
  border-radius: 8px;
  border-bottom: 4px solid #C24648;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  font-family: 'Oxygen', sans-serif;
  letter-spacing: 2px;
}

button:active {
  background-color: #C24648;
  border: none;
  border-radius: 8px;
  border-bottom: 4px solid #C24648;
  color: #CCC;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  font-family: 'Oxygen', sans-serif;
  letter-spacing: 2px;
}

#authorization {
	max-width: 600px;
	width: 70%;
	margin: 100px auto;
	border: 1px solid black;
}

input {
	display: block;
	margin: 10px auto;
	padding: 5px;
	font-size: 16px;
	border-radius: 5px;
}

button, input[type="submit"] {
	margin: 20px;
	padding: 7px 35px;
	border-radius: 10px;
	font-size: 16px;
	cursor: pointer;
	display: inline-block;
}

### Components Interacting

#### CodyOverFlow Forum
In this project, we will build the beginnings of a web forum! A forum is an online discussion board where users can exchange their opinions on a variety of topics. The most important component of a forum is the comment section. This is what we’ll work on in this project.

In building this application, you will sharpen your skills in React components by practicing using props as well as defining, rendering, and referencing components.

If you get stuck during this project or would like to see an experienced developer work through it, click “Get Unstuck“ to see a project walkthrough video.

Let’s get started!

**Tasks**
**Setting Up Header and Body**
1. Take a look at the files provided for this project.

There is a file named **commentData.js** which contains an array of comment objects with `profileImg`, `username`, and `comment` as its properties. You will use this to populate the comments on the forum.

Your comment section will be composed of `Card` components, which will be comprised of smaller `Header` and `Body` child components.

Next, **App.js** will contain the top-level component. The data in **commentData.js** will be retrieved in **App.js** and flow downward from `App` to `Card` to `Header` and `Body`.

**index.js** will render the `App` component. **style.css** and **index.html** handle the markup and styling of the forum.

Once you feel comfortable with the structure of this application, move on to the next checkpoint.


2. Let’s start with the smallest component, the `Body`. The `Body` is responsible for showing the comments that users have written. It will work with the `comment` property of our passed-down `comments` object extracted from **commentData.js**.

Open up **Body.js** and define your `Body` component.

This component should receive `props`. This `props` object contains all of the data on our comment, such as the username, profile image, and comment, but the `Body` works specifically with only the `comment` property.

Have the component return a `<p>` element that contains the `comment` property from `props`.


3. Export the `Body` component after its function definition so that it can be imported and used in **Card.js**.


4. Open **Header.js** and define your `Header` component.

This component will be responsible for the `profileImg` and `username` properties of our passed-down `comments` object.

The `Header` component should receive `props`. The component should return two elements, an `<img>` element whose `src` attribute will receive the `profileImg` property from `props`, and a `<h1>` element, displaying `username` from `props`.


5. Export the `Header` component after its function definition so that it can be imported and used in **Card.js**.


**Setting Up Card**
6. Great job! Let’s take a brief detour back to **App.js**.

Currently, the `App` component is empty. This is the top-level component and it is responsible for returning the interface to be rendered. In this case, it will be returning an instance of the `Card` component for every comment in **commentData.js**.

In the `App` component body, map over the `comments` array with the argument named `comment` and return an instance of the `Card` component.

For each component, give it an attribute named `commentObject` and the value `{comment}`.


7. Nice! You’ve set up `App` to pass down information to the `Card` component. Let’s put the pieces together.

Open **Card.js** and import the `Header` and `Body` components. `Card` will be the outer “shell” that contains our two smaller components: `Card` is the parent, and `Header` and `Body` are the child components.

Begin defining the `Card` component. `Card` should receive `props`.

Card should return an instance of the Header component, and an instance of the Body component.


8. Previously in **App.js**, you’ve passed down a `commentObject` attribute to `Card` which contains an object with three properties (`profileImg`, `username`, and `comment`).

It’s time to access those and finally pass them to the children components: `Header` and `Body`.

`Header` expects two props, `profileImg` and `username`.

Give the `Header` instance an attribute named `profileImg` and the value of `props.commentObject.profileImg`.


9. Next, give the `Header` instance an attribute named `username` and the value of `props.commentObject.username`.


10. The `Body` instance expects a `comment` prop.

Give the `Body` instance an attribute named `comment` and the value of `props.commentObject.comment`.


11. Now, export the `Card` component so it can be imported and used.


**Rendering the Application**
12. Open up **App.js** again, and import the `Card` component which is now defined and exported.


13. Then, export the `App` component.


14. Open up **index.js** and import the `App` component.


15. As you may recall, **index.js** is largely responsible for only rendering the `App` component.

Use `.createRoot()` to supply a container where you will render the `App` component. Then, use `.render()` to render it into the DOM!

If you’ve succeeded, you should see three comments rendered on the web browser, engaging in a riveting discussion about animals.

In [ ]:
<!-- #+name: style.css -->
img {
    border-radius: 50px;
    width: 50px;
    height: 50px;
    float: left;
    margin-right: 10px;
  }
  
  h1 {
    display: inline-block;
    vertical-align: middle;
    line-height: 50px;
    margin: 0;
  }
  
  p {
    margin-left: 70px;
  }

In [ ]:
// #+name: App.js
import React from 'react';
import {comments} from './commentData';
import Card from './Card';

function App() {
  return comments.map(comment => <Card commentObject={comment} />);
}

export default App;

In [ ]:
// #+name: Index.js
import React from 'react';
import {createRoot} from 'react-dom/client';
import App from './App';

const root = createRoot(document.getElementById('app'));
root.render(<App />);

In [ ]:
// #+name: Card.js
import React from 'react';
import Header from './Header';
import Body from './Body';

function Card(props) {
    return (
    <div>
        <Header profileImg={props.commentObject.profileImg} username={props.commentObject.username} />
        <Body comment={props.commentObject.comment} />
    </div>);
}

export default Card;

In [ ]:
// #+name: Header.js
import React from 'react';

function Header(props) {
    return (
        <div>
            <img src={props.profileImg} />
            <h1>{props.username}</h1>
        </div>
    );
}

export default Header;

In [ ]:
// #+name: Body.js
import React from 'react';

function Body(props) {
    return <p>{props.comment}</p>;
}

export default Body;

### Hooks

#### The State Hook

##### Objects in State
We can also use state with objects. When we work with a set of related variables, it can be very helpful to group them into an object. Let’s look at an example of this in action.
```
export default function Login() {
  const [formState, setFormState] = useState({});
  const handleChange = ({ target }) => {
    const { name, value } = target;
    setFormState((prev) => ({
      ...prev,
      [name]: value
    }));
  };
 
  return (
    <form>
      <input
        value={formState.firstName}
        onChange={handleChange}
        name="firstName"
        type="text"
      />
      <input
        value={formState.password}
        onChange={handleChange}
        type="password"
        name="password"
      />
    </form>
  );
}
```

A few things to notice:

- We use a state setter callback function to update a state based on the previous value.

- The spread syntax is the same for objects as for arrays: `{ ...oldObject, newKey: newValue }`.

- We reuse our event handler across multiple inputs by using the input tag’s `name` attribute to identify which input the change event came from.

Once again, when updating the state with `setFormState()` inside a function component, we do not modify the same object. We must copy over the values from the previous object when setting the next value of a state. Thankfully, the spread syntax makes this super easy to do!

Anytime one of the input values is updated, the `handleChange()` function will be called. Inside this event handler, we use object destructuring to unpack the `target` property from our `event` object, then we use object destructuring again to unpack the `name` and `value` properties from the `target` object.

Inside our state setter callback function, we wrap our curly brackets in parentheses like so:
```
setFormState((prev) => ({ ...prev }))
```

This tells JavaScript that our curly brackets refer to a new object to be returned. We use `...`, the spread operator, to fill in the corresponding fields from our previous state. Finally, we overwrite the appropriate key with its updated value.

Did you notice the square brackets around the `name`? This Computed Property Name allows us to use the string value stored by the `name` variable as a property key.


1. We’ll use objects with states to build an input form.

The local state variable profile and state setter function setProfile are responsible for keeping track of the input values from our users. In our JSX, we are looking up properties stored in the profile object. This throws an error at our first render because we are attempting to get the value of a property from an object that has not been defined yet.

To fix this, initialize profile as an empty object.


2. You should now see the form rendered, but nothing will happen when we type in the input boxes. Our form does not re-render to show the keystrokes yet.

To fix this, add the onChange event listener to our JSX tags to call handleChange() whenever a user types in an input field. This way, we can determine what happens when the user changes the input by typing in the form.


3. Let’s make our handleChange() function a bit easier to read. Use object destructuring to initialize name and value in a more concise way.


4. There’s a bug in our code! Have you noticed it? Try typing in one input, then type in a different input. What happens? Why?

Each time that we call setProfile() in our event handler, we give profile the value of a new object with the name and value of the input that most recently changed, but we lose the values that were stored for inputs with any other name.

Use the spread operator to fix this bug. We want to copy over all of the values from our previous profile object whenever we call our state setter function. Use prevProfile as the argument for our state setter callback function.


5. Finally, add an event listener to the <form> tag to call our handleSubmit() function when the user submits the form.

#+name: OriginalEditProfile.js
#+begin_src javascript
import React, { useState } from "react";

export default function EditProfile() {
  const [profile, setProfile] = useState();

  const handleChange = ({ target }) => {
    const name = target.name;
    const value = target.value;
    setProfile({
      [name]: value
    });
  };

  const handleSubmit = (event) => {
    event.preventDefault();
    alert(JSON.stringify(profile, '', 2));
  };

  return (
    <form>
      <input
        value={profile.firstName || ''}
        name="firstName"
        type="text"
        placeholder="First Name"
      />
      <input
        value={profile.lastName || ''}
        type="text"
        name="lastName"
        placeholder="Last Name"
      />
      <input
        value={profile.bday || ''}
        type="date"
        name="bday"
      />
      <input
        value={profile.password || ''}
        type="password"
        name="password"
        placeholder="Password"
      />
      <button type="submit">Submit</button>
    </form>
    
  );
}
#+end_src


#+name: EditProfile.js
#+begin_src javascript
import React, { useState } from "react";

export default function EditProfile() {
  const [profile, setProfile] = useState({});

  const handleChange = ({ target }) => {
    // Use object destructuring to initialize name and value
    const {name, value} = target;

    setProfile((prevProfile) => ({
      // Use the spread operator here (...provProfile)
      ...prevProfile,
      [name]: value
    }));
  };  

  const handleSubmit = (event) => {
    event.preventDefault();
    alert(JSON.stringify(profile, '', 2));
  };

  return (
    <form>
      <input
        value={profile.firstName || ''}
        name="firstName"
        type="text"
        placeholder="First Name"
        onChange={handleChange}
      />
      <input
        value={profile.lastName || ''}
        type="text"
        name="lastName"
        placeholder="Last Name"
        onChange={handleChange}
      />
      <input
        value={profile.bday || ''}
        type="date"
        name="bday"
        onChange={handleChange}
      />
      <input
        value={profile.password || ''}
        type="password"
        name="password"
        placeholder="Password"
        onChange={handleChange}
      />
      <button type="submit" onClick={handleSubmit}>Submit</button>
    </form>
    
  );
}
#+end_src

#+name: Index.js
#+begin_src javascript
import React from 'react';
import ReactDOM from 'react-dom/client';

import App from './App.js';

ReactDOM.createRoot( 
  document.querySelector('#app')
).render(<App />)
#+end_src

#### The Effect Hook
##### Why Use useEffect?
Before Hooks, function components were only used to accept data in the form of props and return some JSX to be rendered. However, as we learned in the last lesson, the State Hook allows us to manage dynamic data, in the form of component state, within our function components.

In this lesson, we’ll use the **Effect Hook** to run some JavaScript code after each render to:

- fetch data from a back-end service.
- subscribe to a stream of data.
- manage timers and intervals.
- read from and make changes to the DOM.

Components will re-render multiple times throughout their lifetime. These key moments present the perfect opportunity to execute these “side effects”.

There are three key moments when the Effect Hook can be utilized:

1. When the component is first added, or *mounted*, to the DOM and renders.
2. When the state or props change, causing the component to re-render.
3. When the component is removed, or *unmounted*, from the DOM.

Later on in this lesson, we’ll learn how to further fine-tune exactly when the Effect Hook executes.

##### Function Component Effects
The Effect Hook tells our component to do something every time it’s rendered (or re-rendered). Combined with states, we can use the Effect Hook to create interesting dynamic changes in our web pages!

Suppose we want to allow a user to change the title of the web page tab every time they type. We can implement this with the Effect Hook (useEffect()) like so:
```
import React, { useState, useEffect } from 'react';
 
function PageTitle() {
  const [name, setName] = useState('');
 
  useEffect(() => {
    document.title = `Hi, ${name}`;
  });
 
  return (
    <div>
      <p>Use the input field below to rename this page!</p>
      <input onChange={({target}) => setName(target.value)} value={name} type='text' />
    </div>
  );
}
```

Let’s take a look at the above example in more detail. First, we import the Effect Hook from the `'react'` library:

```
import { useEffect } from 'react';
```

The `useEffect()` function has no return value as the Effect Hook is used to call another function. We pass the callback function, or *effect*, to run after a component renders as the argument of the `useEffect()` function. In our example, the following effect runs after each time the `PageTitle` component renders:

```
() => { document.title = `Hi, ${name}`;}
```

Here, we assign `Hi, ${name}` as the value of `document.title`.

The `onChange` event listener triggers the `PageTitle` component to be re-rendered every time the user types in the input. Consequently, this triggers `useEffect()` and changes the document’s title.

Notice how we use the current state inside of our effect. Even though our effect is called after the component renders, we still have access to the variables in the scope of our function component! When React renders our component, it will update the DOM as usual, and then run our effect after the DOM has been updated. This happens for every render, including the first and last one.


1. Import the Effect Hook, the State Hook, and React from the 'react' library.

Make sure to import everything in one line.


2. Call useEffect() with a callback function that creates an alert with the current value of count.

Start clicking the button to see when our alert() function is called and be sure that it is logging the values that we’d expect!


3. Use a template literal so that the message in our alert dialog reads: “Count: 0”, then “Count: 1”, then “Count: 2”, etc.

#+name: Counter.js
#+begin_src
import React, { useState, useEffect } from 'react';

export default function Counter() {
  const [count, setCount] = useState(0);
  useEffect(() => {
    alert(`Count: ${count}`);
  })
  const handleClick = () => {
    setCount((prevCount) =>  prevCount + 1);
  };

  return (
    <div>
      <p>You clicked {count} times</p>
      <button onClick={handleClick}>
        Click me
      </button>
    </div>
  );
}
#+end_src

##### Clean Up Effects
Some effects require **cleanup**. For example, we might want to add event listeners to some element in the DOM, beyond the JSX in our component. When we add event listeners to the DOM, it is important to remove those event listeners when we are done with them to avoid memory leaks!

Let’s consider the following effect:

```
useEffect(()=>{
  document.addEventListener('keydown', handleKeyPress);
  // Specify how to clean up after the effect:
  return () => {
    document.removeEventListener('keydown', handleKeyPress);
  };
})
```

If our effect didn’t return a *cleanup function*, a new event listener would be added to the DOM’s `document` object every time that our component re-renders. Not only would this cause bugs, but it could cause our application performance to diminish and maybe even crash!

Because effects run after every render and not just once, React calls our cleanup function before each re-render and before unmounting to clean up each effect call.

If our effect returns a function, then the `useEffect()` Hook always treats that as the cleanup function. React will call this cleanup function before the component re-renders or unmounts. Since this cleanup function is optional, it is our responsibility to return a cleanup function from our effect when our effect code could create memory leaks.


1. Let’s create a program that documents how many times you’ve clicked on the page.

Write an event handler named increment() that will be responsible for tracking how many times a user has clicked. Define this function so that it calls setClickCount() with a state setter callback function, adding 1 to the previous value of clickCount.


2. Import the useEffect() hook and call it with an effect that adds an event listener for 'mousedown' events on the document object. When a 'mousedown' event occurs anywhere on the document, we want our increment() event handler to be called.


3. If you haven’t already, run our code and click around the browser window. What is happening? Why is this happening?

Each time that our component renders, our effect is called, adding another event listener. With just a few clicks and rerenders, we have attached a lot of event listeners to the DOM! We need to clean up after ourselves!

Update our effect so that it returns a cleanup function that will remove our last event listener from the DOM.

In [ ]:
// Counter.js
import React, { useState, useEffect } from 'react';

export default function Counter() {
  const [clickCount, setClickCount] = useState(0);

  // your code here
  // When a 'mousedown' event occurs anywhere on the document, we want our increment() event handler to be called
  useEffect(() => {
    document.addEventListener('mousedown', increment);

    // return a cleanup function that will remove our last event listener from the DOM
    return () => {
      document.removeEventListener('mousedown', increment);
    };
  });

  // an event handler named increment() that will be responsible for tracking how many times a user has clicked. 
  // Define this function so that it calls setClickCount() with a state setter callback function,
  // adding 1 to the previous value of clickCount
  const increment = () => {
  setClickCount((prevClickCount) => prevClickCount + 1)
};

  return (
      <h1>Document Clicks: {clickCount}</h1>
  );
}

##### Control When Effects Are Called
The `useEffect()` function calls its first argument (the effect) after each time a component renders. We’ve learned how to return a cleanup function so that we don’t create performance issues and other bugs, but sometimes we want to skip calling our effect on re-renders altogether.

It is common, when defining function components, to run an effect only when the component mounts (renders the first time), but not when the component re-renders. The Effect Hook makes this very easy for us to do! If we want to only call our effect after the first render, we pass an empty array to `useEffect()` as the second argument. This second argument is called the **dependency array**.

The dependency array is used to tell the `useEffect()` method when to call our effect and when to skip it. Our effect is always called after the first render but only called again if something in our dependency array has changed values between renders.

We will continue to learn more about this second argument over the next few exercises, but for now, we’ll focus on using an empty dependency array to call an effect when a component first mounts, and if a cleanup function is returned by our effect, calling that when the component unmounts.
```
useEffect(() => {
  alert("component rendered for the first time");
  return () => {
    alert("component is being removed from the DOM");
  };
}, []); 
```
Without passing an empty array as the second argument to the `useEffect()` above, those alerts would be displayed before and after every render of our component, which is clearly not when those messages are meant to be displayed. Simply passing `[]` to the `useEffect()` function is enough to configure when the effect and cleanup functions are called!



1. Let’s get started by using the following four functions to advance the number stored in time each second:

- useEffect(): the Effect Hook, imported from the ‘react’ library.
- JavaScript setInterval() function.
- setTime(): our state setter function.
- A state setter callback function: used by setTime() to calculate the next value of time based on the previous value of time.
- Add an effect that uses the setInterval() function to call setTime() every second (or 1000 ms).


2. Our time value is updating way too quickly because the Effect Hook calls our effect after every render! Our effect is creating a new interval that updates the value of time each second. We keep adding more and more intervals that keep updating the same time variable. We need to clean up our old intervals before adding new ones!

Let’s start by creating a variable, intervalId and assign it to our setInterval() code from the previous step.

Then, below your intervalId declaration, use the return keyword to return a cleanup function. Our cleanup function should use the clearInterval() function.


3. That seems to have solved our way-too-many-intervals-all-updating-the-same-variable bug!

Let’s add an extra variable to our timer and allow the user to type a message while the timer is counting up.

First, let’s create a state variable called name with a state setter called setName() to manage the value of the input box. Set the state variable to the initial value of an empty string.


4. Great! Let’s go ahead and put that input tag in.

Add an <input> element to our JSX. Set its value attribute to our state variable name.


5. Next, define an event handler function named handleChange(). This event handler will take in the value of the user’s input and update the state variable.

handleChange() should use object destructuring on its parameter to take in target and use the state setter setName to set the value of name to target.value.


6. Excellent job! Put that handleChange() event handler to work!

Add the onChange event listener to the input tag, setting it to handleChange().

Try typing now!


7. Uh oh. More bugs. Did you notice it yet? Type your full name in the text input field. See how the timer seems to stop counting while you are typing? That’s not what we want!

What is going on here? We are creating a new interval after each render, that interval will call our state setter to update time exactly one second after each render. When we type in the input field, our component keeps re-rendering, cleaning up old intervals, and starting new ones… but our state setter never gets called until one second after we are done typing!

Let’s fix this once and for all! We really want to use a single interval. We want that interval to start ticking away after our first render and we want it to be cleaned up after the final render.

To accomplish this, use an empty dependency array!

In [ ]:
// OriginalTimer.js
import React, { useState } from 'react';

export default function Timer() {
  const [time, setTime] = useState(0);

  return (
    <>
      <h1>Time: {time}</h1>
    </>
  );
}

In [ ]:
// Timer.js
import React, { useState, useEffect } from 'react';

export default function Timer() {
  const [time, setTime] = useState(0);

  // create a state variable called name
  const [name, setName] = useState('');

  useEffect(() => {
    // create a variable intervalId and assign to setInterval() code
    const intervalId = setInterval(() => {
      setTime((prev) => prev + 1);
    }, 1000);

    // return a cleanup function
    return () => {
      clearInterval(intervalId);
    };
  }, []);

  // event handler to take in user input and update the state variable
  const handleChange = ({target}) => {
    setName(target.value);
  };


  return (
    <>
      <h1>Time: {time}</h1>
      <input value={name} onChange={handleChange} type='text' />
    </>
  );
}

##### Fetch Data
When building software, we often start with default behaviors and then modify them to improve performance.

We’ve learned that the default behavior of the Effect Hook is to call the effect function after every single render.

Next, we learned that we can pass an empty array as the second argument for `useEffect()` if we only want our effect to be called after the component’s first render.

In this exercise, we’ll learn to use the dependency array to further configure exactly when we want our effect to be called!

When our effect is responsible for fetching data from a server, we pay extra close attention to when our effect is called. Unnecessary round trips back and forth between our React components and the server can be costly in terms of:

- Processing
- Performance
- Data usage for mobile users
- API service fees

When the data that our components need to render doesn’t change, we can pass an empty dependency array so that the data is fetched after the first render. When the response is received from the server, we can use a state setter from the State Hook to store the data from the server’s response in our local component state for future renders. Using the State Hook and the Effect Hook together in this way is a powerful pattern that saves our components from unnecessarily fetching new data after every render!

An empty dependency array signals to the Effect Hook that our effect never needs to be re-run, that it doesn’t depend on anything. Specifying zero dependencies means that the result of running that effect won’t change and calling our effect once is enough.

A dependency array that is not empty signals to the Effect Hook that it can skip calling our effect after re-renders unless the value of one of the variables in our dependency array has changed. If the value of a dependency has changed, then the Effect Hook will call our effect again!

Here’s a nice example from the official React docs:
```
useEffect(() => {
  document.title = `You clicked ${count} times`;
}, [count]); // Only re-run the effect if the value stored by count changes
```


``` {javascript OriginalForecast.js}
import React, { useState, useEffect } from "react";
import { get } from './mockBackend/fetch';

export default function Forecast() {
  const [data, setData] = useState();
  const [notes, setNotes] = useState({});
  const [forecastType, setForecastType] = useState('/daily');

  useEffect(() => {
    alert('Requested data from server...');
    get('/daily').then((response) => {
      alert('Response: ' + JSON.stringify(response,'',2));
    });
  });

  const handleChange = (index) => ({ target }) =>
    setNotes((prev) => ({
      ...prev,
      [index]: target.value
    }));

  return (
    <div className='App'>
      <h1>My Weather Planner</h1>
      <div>
        <button onClick={() => setForecastType('/daily')}>5-day</button>
        <button onClick={() => setForecastType('/hourly')}>Today</button>
      </div>
      <table>
        <thead>
          <tr>
            <th>Summary</th>
            <th>Avg Temp</th>
            <th>Precip</th>
            <th>Notes</th>
          </tr>
        </thead>
        <tbody>
          {data.map((item, i) => (
            <tr key={item.id}>
              <td>{item.summary}</td>
              <td> {item.temp.avg}°F</td>
              <td>{item.precip}%</td>
              <td>
                <input
                  value={notes[item.id] || ''}
                  onChange={handleChange(item.id)}
                />
              </td>
            </tr>
          ))}
        </tbody>
      </table>
    </div>
  );
}
```


1. We’ve started building a weather planner app that will fetch data about the weather and allow our users to write some notes next to the forecast. A lot of good code has already been written, but there currently isn’t anything rendering to the screen.

Let’s read through the code and start to wrap our heads around what is going on here. What part of our code do we think is keeping the component from rendering?

In our JSX, we are trying to map over an array stored by the data state variable, but our effect that fetches this data doesn’t get called until after the first render. So during the first render, data is undefined and attempting to call map() on undefined is causing our error!

Let’s prevent this error by checking to see if the data has loaded yet. If it hasn’t, then we want our function component to just return a paragraph tag with the text “Loading…”. If the data is no longer undefined, then the data has been loaded, and we can go ahead and render the full JSX!


2. Our data fetching is being done in our effect. Notice how we are currently just using alert() messages to keep track of requesting and receiving data from our server. Instead of just stringifying the response data and showing it in an alert message, let’s store that data in our state.

When the data has been fetched, use our state setter function to store that data in our component’s state!

Remember that we want to store an array in our data state variable, not the whole response object.


3. Type in each of the notes’ input fields in our table. What do you notice? Why do you think this is happening?

Each time that we type in an input field, the component re-renders to show the new value of that field. Even though we don’t need any new data from the backend, our component is fetching new data after every render!

Use an empty dependency array to ensure that data is only fetched after our component’s first render.


4. Wow, that small code change made a huge difference in the performance of our weather planner app!

Let’s make one more improvement. Did you notice the buttons at the top of our app? We want our users to be able to choose between planning around daily weather forecasts and weekly weather forecasts. Clicking on these buttons currently doesn’t change anything. Let’s fix that!

The server has two different endpoints called: /daily and /hourly. Let’s use the value of the forecastType state variable to determine which endpoint our effect should request data from.

After making this change, our effect behaves differently based on the value of forecastType. You could say that how we use our effect depends on it! Add this variable to our dependency array so that the effect is called again, updating data appropriately, after re-renders where the user has selected a different forecast type.

In [ ]:
// Forecast.js
import React, { useState, useEffect } from "react";
import { get } from './mockBackend/fetch';

export default function Forecast() {
  const [data, setData] = useState();
  const [notes, setNotes] = useState({});
  const [forecastType, setForecastType] = useState('/daily');

  // store response data in the "data" state variable
  useEffect(() => {
    alert('Requested data from server...');
    get(forecastType).then((response) => {
      alert('Response: ' + JSON.stringify(response,'',2));
      setData(response.data);
    });
  }, [forecastType]);

  const handleChange = (index) => ({ target }) =>
    setNotes((prev) => ({
      ...prev,
      [index]: target.value
    }));

  // check if data has loaded. If not, return "Loading..."
  if (!data) {
    return <p>Loading...</p>;
  }

  return (
    <div className='App'>
      <h1>My Weather Planner</h1>
      <div>
        <button onClick={() => setForecastType('/daily')}>5-day</button>
        <button onClick={() => setForecastType('/hourly')}>Today</button>
      </div>
      <table>
        <thead>
          <tr>
            <th>Summary</th>
            <th>Avg Temp</th>
            <th>Precip</th>
            <th>Notes</th>
          </tr>
        </thead>
        <tbody>
          {data.map((item, i) => (
            <tr key={item.id}>
              <td>{item.summary}</td>
              <td> {item.temp.avg}°F</td>
              <td>{item.precip}%</td>
              <td>
                <input
                  value={notes[item.id] || ''}
                  onChange={handleChange(item.id)}
                />
              </td>
            </tr>
          ))}
        </tbody>
      </table>
    </div>
  );
}

##### Rules of Hooks
There are two main rules to keep in mind when using Hooks:

1. Only call Hooks at the top level.
2. Only call Hooks from React functions.

As we have been practicing with the State Hook and the Effect Hook, we’ve been following these rules with ease, but it is helpful to keep these two rules in mind as you take your new understanding of Hooks out into the wild and begin using more Hooks in your React applications.

When React builds the Virtual DOM, the library calls the functions that define our components over and over again as the user interacts with the user interface. React keeps track of the data and functions that we are managing with Hooks based on their order in the function component’s definition. For this reason, we always call our Hooks at the top level; we never call hooks inside of loops, conditions, or nested functions.

Instead of confusing React with code like this:

if (userName !== '') {
  useEffect(() => {
    localStorage.setItem('savedUserName', userName);
  });
}


We can accomplish the same goal while consistently calling our Hook every time:

useEffect(() => {
  if (userName !== '') {
    localStorage.setItem('savedUserName', userName);
  }
});


Secondly, Hooks can only be used in React Functions. We’ve been working with `useState()` and `useEffect()` in *function* components, and this is the most common use. The only other place where Hooks can be used is within custom hooks. Custom Hooks are incredibly useful for organizing and reusing stateful logic between function components.

In [ ]:
// {javascript OriginalShop.js}
import React, { useState, useEffect } from 'react';
import { get } from './mockBackend/fetch';

export default function Shop() {
  const [categories, setCategories] = useState();
  if (categories) {
    const [selectedCategory, setSelectedCategory] = useState();
    const [items, setItems] = useState({});
  }

  if (!categories) {
    useEffect(() => {
      get('/categories').then((response) => {
        setCategories(response.data);
      });
    });
  }

  // if (selectedCategory && !items[selectedCategory]) {
  //   useEffect(() => {
  //     get(`/items?category=${selectedCategory}`).then((response) => {
  //       setItems((prev) => ({ ...prev, [selectedCategory]: response.data }));
  //     });
  //   });
  // }

  if (!categories) {
    return <p>Loading..</p>;
  }

  return (
    <div className='App'>
      <h1>Clothes 'n Things</h1>
      <nav>
        {categories.map((category) => (
          <button key={category} onClick={() => setSelectedCategory(category)}>
            {category}
          </button>
        ))}
      </nav>
      <h2>{selectedCategory}</h2>
      <ul>
        {!items[selectedCategory]
          ? null
          : items[selectedCategory].map((item) => <li key={item}>{item}</li>)}
      </ul>
    </div>
  );
}

1. The code that we are starting with has a lot of good ideas, but there are some bugs that we need to help sort out. Let’s get started by refactoring the code so that the State Hook is always called at the top level.

It looks like the developers that wrote this code wanted to hold off on using the `selectedCategory` and `items` state variables until after the `categories` have been fetched. Conceptually this makes sense, but React requires that all hooks be called on every render, so nesting these `useState()` calls is not a valid option.

First, remove the `if (categories)` statement, and the surrounding curly braces `{ }` to bring all of our State Hook calls to the top level.


2. Next, to be clear about initial values, let’s explicitly set the initial state value for `categories` and `selectedCategory` to `null`.


3. It looks like the idea behind using this expression: `if (!categories)` was to only fetch the categories data from the server once. Nesting a call to the Effect Hook inside of a condition like this will cause different hooks to be called on different re-renders, resulting in errors. Luckily, we know a better way!

Refactor this code so that the effect responsible for fetching the `categories` data from the backend and saving it to local state follows the rules for Hooks and only fetches the categories data once.


4. Whew, we’re making great progress! It’s such a nice feeling to turn error screens into working code, isn’t it?

Now that we are fetching the list of categories from the backend and successfully rendering buttons for each of these to the screen, we are ready to use another effect to fetch the items for each of these categories, when the user clicks on each of them!

Uncomment the block of code that was attempting to do this, and refactor it so that we follow the rules of Hooks. To optimize performance, only call the backend for data when we don’t yet have it stored in the component’s state like this code was trying to do.

In [ ]:
// {javascript Shop.js}
import React, { useState, useEffect } from 'react';
import { get } from './mockBackend/fetch';

export default function Shop() {
  // explicitly set the initial state value for categories and selectedCategory to null
  const [categories, setCategories] = useState(null);

  const [selectedCategory, setSelectedCategory] = useState(null);
  const [items, setItems] = useState({});

  /*
When using the Effect Hook, passing a dependency array as the second argument for useEffect() is the best way to determine when our effect is and is not called.

Remove the if (!categories) condition, and pass an empty dependency array so that this effect is only called after the first render.
  */
  useEffect(() => {
      get('/categories').then((response) => {
        setCategories(response.data);
      });
    }, []);
  
  // list the 2 variables that this effect depends on in the dependency array
  useEffect(() => {
    if (selectedCategory && !items[selectedCategory]) {
      // fetch data and store it to local state
      get(`/items?category=${selectedCategory}`).then((response) => {
        setItems((prev) => (
          { ...prev, [selectedCategory]: response.data }));
        });
}
}, [items, selectedCategory]);


  if (!categories) {
    return <p>Loading..</p>;
  }

  return (
    <div className='App'>
      <h1>Clothes 'n Things</h1>
      <nav>
        {categories.map((category) => (
          <button key={category} onClick={() => setSelectedCategory(category)}>
            {category}
          </button>
        ))}
      </nav>
      <h2>{selectedCategory}</h2>
      <ul>
        {!items[selectedCategory]
          ? null
          : items[selectedCategory].map((item) => <li key={item}>{item}</li>)}
      </ul>
    </div>
  );
}

##### Separate Hooks for Separate Effects
When multiple values are closely related and change at the same time, it can make sense to group these values in a collection like an object or array. Packaging data together can also add complexity to the code responsible for managing that data. Therefore, it is a good idea to separate concerns by managing different data with different Hooks.

Compare the complexity here, where data is bundled up into a single object:

```
// Handle both position and menuItems with one useEffect hook.
const [data, setData] = useState({ position: { x: 0, y: 0 } });
useEffect(() => {
  get('/menu').then((response) => {
    setData((prev) => ({ ...prev, menuItems: response.data }));
  });
  const handleMove = (event) =>
    setData((prev) => ({
      ...prev,
      position: { x: event.clientX, y: event.clientY }
    }));
  window.addEventListener('mousemove', handleMove);
  return () => window.removeEventListener('mousemove', handleMove);
}, []);
```

To the simplicity here, where we have separated concerns:

```
// Handle menuItems with one useEffect hook.
const [menuItems, setMenuItems] = useState(null);
useEffect(() => {
  get('/menu').then((response) => setMenuItems(response.data));
}, []);
 
// Handle position with a separate useEffect hook.
const [position, setPosition] = useState({ x: 0, y: 0 });
useEffect(() => {
  const handleMove = (event) =>
    setPosition({ x: event.clientX, y: event.clientY });
  window.addEventListener('mousemove', handleMove);
  return () => window.removeEventListener('mousemove', handleMove);
}, []);
```

It is not always obvious whether to bundle data together or separate it, but with practice, we get better at organizing our code so that it is easier to understand, add to, reuse, and test!



**Instructions**
1. At the moment, this code seems to work just fine. There are three different network requests being made in a single effect and lots of different, unrelated data being managed in a single state variable. Let’s get to work breaking these single `useState()` and `useEffect()` calls into separate, simpler hooks. Doing so will make this code easier to understand, build on top of, and reuse as we continue to improve our application!

Begin refactoring this component:

- Use a separate State Hook for `menu`, `newsFeed`, and `friends`.
- Use these new state setters instead of `setData()` in the effect.
- Simplify our JSX to use these new state variables instead of `data`.


2. `Promise.all()` was helpful to us when we had all of our data bundled up in a single object. It called all backend services, and when they all sent back responses, we could then call our state setters with the responses.

Because we are now managing these values separately, we can remove this complexity! In our effect, call the `get()` function three times for the three different data collections that our component wants to render, without using `Promise.all()` any more.


3. Now that we have three separate backend calls, let’s continue to separate concerns by splitting each of these into three separate Effect Hooks!


4. Each `useEffect()` call is working with a corresponding `useState()` call. Let’s reorganize our code to show this relationship more clearly, making the logic easier to read and reuse!

For each of these three data collections, group the related State Hook and the Effect Hook next to one. This will help to make it clear which Hooks are working together to manage each separate data model.

In [ ]:
// #+name: OriginalSocialNetwork.js
import React, { useState, useEffect } from 'react';
import { get } from './mockBackend/fetch';

export default function SocialNetwork() {
  const [data, setData] = useState(null);

  useEffect(() => {
    Promise.all([get('/menu'), get('/news-feed'), get('/friends')]).then(
      ([menuResponse, newsFeedResponse, friendsResponse]) => {
        setData({
          menu: menuResponse.data,
          newsFeed: newsFeedResponse.data,
          friends: friendsResponse.data
        });
      }
    );
  }, []);

  return (
    <div className='App'>
      <h1>My Network</h1>
      {!data || !data.menu ? <p>Loading..</p> : (
        <nav>
          {data.menu.map((menuItem) => (
            <button key={menuItem}>{menuItem}</button>
          ))}
        </nav>
      )}
      <div className='content'>
        {!data || !data.newsFeed ? <p>Loading..</p> : (
          <section>
            {data.newsFeed.map(({ id, title, message, imgSrc }) => (
              <article key={id}>
                <h3>{title}</h3>
                <p>{message}</p>
                <img src={imgSrc} alt='' />
              </article>
            ))}
          </section>
        )}
        {!data || !data.friends ? <p>Loading..</p> : (
          <aside>
            <ul>
              {data.friends
                .sort((a, b) => (a.isOnline && !b.isOnline ? -1 : 0))
                .map(({ id, name, isOnline }) => (
                  <li key={id} className={isOnline ? 'online' : 'offline'}>
                    {name}
                  </li>
                ))}
            </ul>
          </aside>
        )}
      </div>
    </div>
  );
}

In [ ]:
// #+name: SocialNetwork.js
import React, { useState, useEffect } from 'react';
import { get } from './mockBackend/fetch';

export default function SocialNetwork() {
  // const [data, setData] = useState(null);
  
  // Use the new state setter instead of setData in the effect
  // Each API call is its own effect, so place each in its own useEffect() call

  // menu
  // separate State Hooks for each state variable
  const [menu, setMenu] = useState(null);
  useEffect(() => {
    get('/menu').then((response) => {
      setMenu(response.data);
    })
  }, []);


  // news feed
  const [newsFeed, setNewsFeed] = useState(null);
  useEffect(() => {
    get('/news-feed').then((response) => {
      setNewsFeed(response.data);
    })
  }, []);


  // friends
  const [friends, setFriends] = useState(null);
  useEffect(() => {
    get('/friends').then((response) => {
      setFriends(response.data);
    })
  }, []);

  // useEffect(() => {
  //   Promise.all([get('/menu'), get('/news-feed'), get('/friends')]).then(
  //     ([menuResponse, newsFeedResponse, friendsResponse]) => {
  //       setData({
  //         menu: menuResponse.data,
  //         newsFeed: newsFeedResponse.data,
  //         friends: friendsResponse.data
  //       });
  //     }
  //   );
  // }, []);

  return (
    <div className='App'>
      <h1>My Network</h1>
      {!menu ? <p>Loading..</p> : (
        <nav>
          {menu.map((menuItem) => (
            <button key={menuItem}>{menuItem}</button>
          ))}
        </nav>
      )}
      <div className='content'>
        {!newsFeed ? <p>Loading..</p> : (
          <section>
            {newsFeed.map(({ id, title, message, imgSrc }) => (
              <article key={id}>
                <h3>{title}</h3>
                <p>{message}</p>
                <img src={imgSrc} alt='' />
              </article>
            ))}
          </section>
        )}
        {!friends ? <p>Loading..</p> : (
          <aside>
            <ul>
              {friends
                .sort((a, b) => (a.isOnline && !b.isOnline ? -1 : 0))
                .map(({ id, name, isOnline }) => (
                  <li key={id} className={isOnline ? 'online' : 'offline'}>
                    {name}
                  </li>
                ))}
            </ul>
          </aside>
        )}
      </div>
    </div>
  );
}

##### Review
In this lesson, we learned how to write effects that manage timers, manipulate the DOM, and fetch data from a server. With the Effect Hook, we can perform these types of actions in function components with ease!

Let’s review the main concepts from this lesson:

- We can import the `useEffect()` function from the `'react'` library and call it in our function components.
    - *Effect* refers to a function that we pass as the first argument of the `useEffect()` function. By default, the Effect Hook calls this effect after each render.

    - The *cleanup function* is optionally returned by the effect. If the effect does anything that needs to be cleaned up to prevent memory leaks, then the effect returns a cleanup function, then the Effect Hook will call this cleanup function before calling the effect again as well as when the component is being unmounted.

    - The *dependency array* is the optional second argument that the `useEffect()` function can be called with in order to prevent repeatedly calling the effect when this is not needed. This array should consist of all variables that the effect depends on.

The Effect Hook is all about scheduling when our effect’s code gets executed. We can use the dependency array to configure when our effect is called in the following ways:

|**Dependency Array**	|**Effect called after first render & ...**|
|-----------------------|------------------------------------------|
|undefined	            |every re-render                           |
|Empty array	        |no re-renders                             |
|Non-empty array	    |when any value in the dependency array changes|

Hooks gives us the flexibility to organize our code in different ways, grouping related data as well as separating concerns to keep code simple, error-free, reusable, and testable!

#### Passing Thoughts
What if you could post something and know it wouldn’t live forever? In this project, we’ll build a place for our passing thoughts. Once you add a short thought, it’ll disappear after just 15 seconds.


**Adding Thoughts**
1. Run the app and take a look at what we have so far.

The app’s structure is there, but…well, it doesn’t actually work! There are three major missing pieces right now:

    1. If you try to add a new thought, it refreshes the whole page and doesn’t do anything.
    
    2. If you try to delete a thought manually, it crashes!
    
    3. Thoughts never disappear, defeating the whole point of the app.

Before we start, let’s talk about the main piece of state that the app stores: an array of thought objects. Each of those objects will have three properties:

- `id`, a unique ID for this thought
- `text`, the thought’s text
- `expiresAt`, the time that the thought expires, represented as a number

Take a look at **App.js**. You’ll see that we’re defining two starter thoughts on lines 8-19 like below:
```
const [thoughts, setThoughts] = useState([
  {
    id: generateId(),
    text: 'This is a place for your passing thoughts.',
    expiresAt: getNewExpirationTime(),
  },
  {
    id: generateId(),
    text: "They'll be removed after 15 seconds.",
    expiresAt: getNewExpirationTime(),
  },
]);
```

That will set the state to something like this:
```
[
  {
    id: 0,
    text: 'This is a place for your passing thoughts.',
    expiresAt: 1600624968405,
  },
  {
    id: 1,
    text: "They'll be removed after 15 seconds.",
    expiresAt: 1600624968405,
  },
]
```

When we create new thought objects, we’ll do something very similar to this. Let’s get started!


2. Open the `App` component in **App.js**. You can see that we’ve already set up the array of thoughts in a variable called `thoughts`, using the `useState()` hook. We’ve also got `setThoughts()`, a function we can call to update the list of thoughts. Let’s wire these things up so that users can add new thoughts.

First, we’ll need to write a new function inside of `App()`, called `addThought()`. It will take a single argument, `thought`, and it will put itself inside of the thoughts array with `setThoughts()`.

Inside of `addThought()`, we’ll call `setThoughts()` with a function that returns a new state: the array with the new `thought` at the front.

Add this function between the end of the `useState()` call (the line ending with `]);`) and the `return`.


3. Now we have a function that’ll update the state, but we need to use it. Specifically, let’s pass it to the `AddThoughtForm` component as a prop.

In **App.js**, find where <AddThoughtForm> is rendered. Pass a prop named `addThought` and give our newly created `addThought()` function as its value.


4. Now that we’re passing `addThought()` into the `AddThoughtForm` component, it’s time for us to make that component actually call this function.

`AddThoughtForm` will have some state. Specifically, it’ll hold the value of the text input, and when the user submits the form, we’ll take that input and use it in a call to `addThought()`.

Open up **AddThoughtForm.js**.

On the first line, we’ll need to import `useState()` and React. Update the import to import both `React` and `useState()`.

Next, inside of `AddThoughtForm`, set up the initial state of the text input as an empty string.

After you have done that, you’ll need to write a function called `handleTextChange()` that will be called when the input changes. It will take the event as an argument, and will call `setText()` to update the state.

Finally, you’ll need to connect these two things to the input. Pass `text` in a prop called `value`, and pass `handleTextChange()` in a prop called `onChange`.

Now we’re storing the state of the input!


5. Now that we have the input’s state stored, we will need to create a new thought object when the form is submitted. Handling the form submission event is the first step of doing that.

Create a new function to handle the form submission called `handleSubmit()`. It will take the event as an argument (just like how `handleTextChange()` does).

To prevent the form from refreshing the page, call `event.preventDefault()` inside `handleSubmit()`. This prevents the browser from performing its default behavior when a form is submitted.

Finally, add the submit handler to the form by passing `onSubmit={handleSubmit}`.

Once you’ve done this, the form should no longer refresh the page. (Nothing else will happen either, but we’ll fix that soon.)


6. Now, we’ll update `handleSubmit()` to…well, actually submit the data!

In **utilities.js** you will see that there are two functions: `generateId()` and `getNewExpirationTime()`. We will use these functions to get the values for the unique ID and the expiration time for new thought objects.

Back in **AddThoughtForm.js**, inside of `handleSubmit()` after the call to `event.preventDefault()`, create a new thought object with its three required properties: `id` (generated by `generateId()`), `text` (from prior steps), and `expiresAt` (generated by `getNewExpirationTime()`). Pass it to `addThought()`.

If you’re not sure how to create a thought object, refer to how it’s done in **App.js**.

Once this is done, you should be able to submit the form and see the new thought appear on the screen!


7. Though we are creating new thoughts, you might notice a piece of the user experience that feels a little unintuitive: the input isn’t cleared when you submit the form. That means that whatever you typed stays around, even though it probably shouldn’t.

Clear the input’s text after adding a new thought.


8. There’s just one thing left to do here: if the user hasn’t typed anything but they submit the form anyway, an empty thought will be created. We all have empty thoughts from time to time, but we probably don’t want to add those to our app.

To fix this, only call `addThought()` if the user has typed something in the text box. You’ll use an `if` statement to check the length of the `text` variable before creating and adding a new thought object. (Make sure to always call `event.preventDefault()`, though, even if the user hasn’t typed anything.)


**Manually Deleting Thoughts**
9. The app should feature two ways to delete thoughts:

    1. Manual deletion, when the user clicks the delete button.

    2. Automatic deletion after 15 seconds.

Let’s start with the first task. It will help us build the scaffolding for the second.

Open **App.js**.

Just like we added a function to add new thoughts, we will need to create a function to remove them, too.

Under `addThought()`, create a new function called `removeThought()`. It will take the ID of the thought we want to remove in an argument called `thoughtIdToRemove`, and it will call `setThoughts()` to remove the thought.

To do this, you’ll call `thoughts.filter()` to filter out the thought we want to remove.


10. Once this is done, all you’ll need to do is pass this new function as a prop to the `Thought` component. Add a new prop called `removeThought` with the newly created `removeThought()` function as its value.

If you’ve done this successfully, you should now be able to manually remove thoughts by clicking the delete button next to each thought.


**Letting Thoughts Drift Away**
11. The app is getting close, but we’re still missing the core feature: making thoughts disappear.

When a `Thought` component is rendered, we want to start a countdown. Once the countdown expires, we want to call `removeThought()`. We’ll do this with an effect hook.

Open **Thought.js**.

Start by importing `useEffect()` from React.


12. Let’s start by setting a timer in the `Thought` component with `setTimeout()`.

Using the `useEffect()` hook, call `setTimeout()`. The first argument should be a function that calls `alert('Time has passed!')` (or any sample text you want—we’ll remove it later). It should happen when the thought expires. You might calculate that like this:
```
const timeRemaining = thought.expiresAt - Date.now();
```

Make sure you remember two things:

    1. Return a function that clears the timeout when you’re done! It’s always good practice to clean up your effects.
    
    2. Add `[thought]` as a dependency, in the second argument to `useEffect()`. You want to re-run the effect every time the `thought` is different.

Once this is done, you should see “Time has passed!” alerts after 15 seconds.


13. There’s just one thing left to do: replace that `alert()` with a call to `removeThought()`. This should already be passed in as `props`, so you’ll just need to delete the `alert()` line and replace it with a call to `removeThought()`.

You’ll need to call `removeThought()` with this thought’s ID, which you can get with `thought.id`.

Once that’s done, try adding some thoughts and watch them disappear…it’s just like your brain, but you built it with React.

In [ ]:
<!-- style.css -->
body {
    margin: 0;
    font-size: 16pt;
    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', 'Roboto', 'Oxygen',
      'Ubuntu', 'Cantarell', 'Fira Sans', 'Droid Sans', 'Helvetica Neue',
      sans-serif;
    -webkit-font-smoothing: antialiased;
    -moz-osx-font-smoothing: grayscale;
  }
  
  input,
  button {
    font: inherit;
    box-sizing: border-box;
  }
  
  input[type='submit'],
  button {
    cursor: pointer;
  }
  
  .App {
    margin: 0 auto;
    max-width: 600px;
    width: 95%;
  }
  
  .App h1:before {
    content: '💭';
    margin-right: 0.5em;
  }
  
  .AddThoughtForm {
    display: flex;
    flex-wrap: wrap;
  }
  
  .AddThoughtForm input {
    padding: 1.5rem;
    border: 1px solid #666;
    border-radius: 3px;
  }
  
  .AddThoughtForm input[type='text'] {
    flex-grow: 1;
    margin-right: 0.5rem;
  }
  
  .thoughts {
    list-style-type: none;
    padding: 0;
  }
  
  .Thought {
    padding: 1rem;
    margin-bottom: 1rem;
    border-radius: 3px;
    background-color: #eee;
  }
  
  .Thought .remove-button {
    font-size: 80%;
    float: right;
    border: 0;
    border-radius: 3px;
    background: transparent;
  }
  
  .Thought .remove-button:hover {
    color: #fff;
    background-color: #000;
  }  

In [ ]:
// #+name: utilities.js
// These functioins gets the values for the unique ID and the expiration time for new thought objects

export function getNewExpirationTime() {
    return Date.now() + 15 * 1000;
  }
  
  let nextId = 0;
  export function generateId() {
    const result = nextId;
    nextId += 1;
    return result;
  }

In [ ]:
// #+name: Thought.js
import React, { useEffect } from 'react';

export function Thought(props) {
  const { thought, removeThought } = props;

  const handleRemoveClick = () => {
    removeThought(thought.id);
  };


  // useEffect() hook. Call setTimeout().
  /*
  The global setTimeout() method sets a timer which executes a function or specified piece of code once the timer expires.

  setTimeout(code, delay)
  setTimeout(functionRef, delay)

functionRef
A function to be executed after the timer expires.

code
An alternative syntax that allows you to include a string instead of a function, which is compiled and executed when the timer expires. This syntax is not recommended for the same reasons that make using eval() a security risk.

delay (Optional)
The time, in milliseconds that the timer should wait before the specified function or code is executed. If this parameter is omitted, a value of 0 is used, meaning execute "immediately", or more accurately, the next event cycle.


The returned timeoutID is a positive integer value which identifies the timer created by the call to setTimeout(). This value can be passed to clearTimeout() to cancel the timeout.

Timeouts are cancelled using clearTimeout().
  */
  useEffect(() => {
    // How long till thought expires
    const timeRemaining = thought.expiresAt - Date.now();

    // 12. alert after 15seconds using setTimeout function
    // 13. replace alert() with a call to removeThought(), passed in as props
    const timeout = setTimeout(() => {
      //alert('Time has passed!');
      removeThought(thought.id);
    }, timeRemaining);

    // clearTimeout when finished
    return () => {
      clearTimeout(timeout);
    };

    // runs this effect everytime the `thought` is different by using [thought] as the dependency array
  }, [thought]);


  return (
    <li className="Thought">
      <button
        aria-label="Remove thought"
        className="remove-button"
        onClick={handleRemoveClick}
      >
        &times;
      </button>
      <div className="text">{thought.text}</div>
    </li>
  );
}

In [ ]:
// #+name: AddThoughtForm.js
import React, { useState } from 'react';
import { generateId, getNewExpirationTime } from './utilities';

export function AddThoughtForm(props) {
  // Set up the initial state of the text input as an empty string
  const [text, setText] = useState('');
  
  // Function that will be called when the input changes
  // Will call setText() to update the state
  function handleTextChange(event) {
    setText(event.target.value);
  };

  // Need to create a new thought object when the form is submitted
  // Create a function to handle the form submission
  const handleSubmit = (event) => {
    // prevent page refresh
    event.preventDefault();
    
    // New thought object with its three required properties
    const thought = {
      id: generateId(),
      text: text,
      expiresAt: getNewExpirationTime()
    };

    // Pass to addThought() if the thought's text.length > 0
    if (text.length > 0) {
      props.addThought(thought);
    }

    // Clear the input’s text after adding a new thought
    setText('');
    
  };
  
  // 4. Pass text in a prop called value, and pass handleTextChange() in a prop called onChange.
  // 5. Add the submit handler to the form
  return (
    <form className="AddThoughtForm" onSubmit={handleSubmit}>
      <input
        type="text"
        aria-label="What's on your mind?"
        placeholder="What's on your mind?"
        value={text}
        onChange={handleTextChange}
      />
      <input type="submit" value="Add" />
    </form>
  );
}

In [ ]:
// #+name: App.js
import React, { useState } from 'react';
import { generateId, getNewExpirationTime } from './utilities';

export function AddThoughtForm(props) {
  // Set up the initial state of the text input as an empty string
  const [text, setText] = useState('');
  
  // Function that will be called when the input changes
  // Will call setText() to update the state
  function handleTextChange(event) {
    setText(event.target.value);
  };

  // Need to create a new thought object when the form is submitted
  // Create a function to handle the form submission
  const handleSubmit = (event) => {
    // prevent page refresh
    event.preventDefault();
    
    // New thought object with its three required properties
    const thought = {
      id: generateId(),
      text: text,
      expiresAt: getNewExpirationTime()
    };

    // Pass to addThought() if the thought's text.length > 0
    if (text.length > 0) {
      props.addThought(thought);
    }

    // Clear the input’s text after adding a new thought
    setText('');
    
  };
  
  // 4. Pass text in a prop called value, and pass handleTextChange() in a prop called onChange.
  // 5. Add the submit handler to the form
  return (
    <form className="AddThoughtForm" onSubmit={handleSubmit}>
      <input
        type="text"
        aria-label="What's on your mind?"
        placeholder="What's on your mind?"
        value={text}
        onChange={handleTextChange}
      />
      <input type="submit" value="Add" />
    </form>
  );
}

### React Programming Patterns
#### Separate Container Components From Presentational Components
As you continue building your React applications, you will soon realize that one component has too many responsibilities and is too difficult to maintain. In this lesson, you will learn a programming pattern that will help organize your React code.

If a component has to have state, make calculations based on props, or manage any other complex logic, then that component shouldn’t also have to render JSX.

To help reduce the component’s complexity, we can break it down into multiple simpler components. How should you break it apart?

The pattern we’ll learn about focuses on splitting complex components into stateful (container) and stateless (presentational) components, where stateful components manage complex state or logic and stateless components only render JSX.

As we progress through this lesson, we’ll take a look at how to apply this pattern to our example React app to break down a complex component into container and presentational components.

#### Create Container Component
Separating container components from presentational components is a popular React programming pattern.

The functional part of a component (maintaining a state, making calculations based on props, etc.) can be separated into a container component, also known as a stateful component.

This container component will be in charge of maintaining the state (creating and updating) and passing it on (we’ll see this later) to any component it renders through props.



1. Click Run. You are looking at a rendered <GuineaPigs /> component.

<GuineaPigs />‘s job is to render a photo carousel of guinea pigs and a dropdown to pick your favorite guinea pig. It does this perfectly well! And yet, it has a problem: it does too much stuff. How might we divide this into container and presentational components?


2. Let’s start separating the logic from the GuineaPigs component into a new container component called GuineaPigsContainer.

First, highlight the entire contents of components/GuineaPigs.js, and copy it to the clipboard. Click on the folder icon near the top left of the code editor. Inside the containers/ folder, create a new file named GuineaPigsContainer.js.

Open containers/GuineaPigsContainer.js, click inside the empty file, and paste. containers/GuineaPigsContainer.js and components/GuineaPigs.js should be identical.

Finally, click Run.

In [ ]:
// #+name: OriginalGuineaPigs.js
import React, { useState, useEffect } from "react";
import { createRoot } from "react-dom/client";

const GUINEAPATHS = [
  "https://content.codecademy.com/courses/React/react_photo-guineapig-1.jpg",
  "https://content.codecademy.com/courses/React/react_photo-guineapig-2.jpg",
  "https://content.codecademy.com/courses/React/react_photo-guineapig-3.jpg",
  "https://content.codecademy.com/courses/React/react_photo-guineapig-4.jpg",
];

function GuineaPigs() {
  const [currentGP, setCurrentGP] = useState(0);
	const [favoriteGP, setFavoriteGP] = useState(0);
	const src = GUINEAPATHS[currentGP];

  const favoriteChangeHandler = (event) => {
    setFavoriteGP(parseInt(event.target.value));
  }

  const resetFavoriteHandler = () => {
    setFavoriteGP(0);
  }

  useEffect(() => {
		const intervalId = setInterval(() => {
			setCurrentGP(prevGP => {
				const nextGP = prevGP + 1;
				return nextGP % GUINEAPATHS.length;
			});
		}, 5000)
		return () => clearInterval(intervalId);
	}, []);


	return (
    <>
      <div data-testid="guineaPigsSlideShow" id="guineaPigsSlideShow">
        <h1>Cute Guinea Pigs</h1>
        <img alt="Guinea Pigs Slideshow" src={src} className={currentGP === favoriteGP? "favorite" : ""}/>
      </div>
		  <div data-testid="guineaPigsForm" id="guineaPigsForm">	
			  <label>Choose Your Favorite Guinea Pig:
          <select value={favoriteGP} onChange={favoriteChangeHandler}>
            <option value="0">Alex</option>
            <option value="1">Izzy</option>
            <option value="2">Brandon</option>
            <option value="3">DJ</option>
          </select>
        </label>
        <button onClick={resetFavoriteHandler}>Reset Favorite</button>
		  </div>
  </>
  );
}

export default GuineaPigs;

In [ ]:
// #+name: containers/GuineaPigsContainer.js
import React, { useState, useEffect } from "react";
import { createRoot } from "react-dom/client";

const GUINEAPATHS = [
  "https://content.codecademy.com/courses/React/react_photo-guineapig-1.jpg",
  "https://content.codecademy.com/courses/React/react_photo-guineapig-2.jpg",
  "https://content.codecademy.com/courses/React/react_photo-guineapig-3.jpg",
  "https://content.codecademy.com/courses/React/react_photo-guineapig-4.jpg",
];

function GuineaPigs() {
  const [currentGP, setCurrentGP] = useState(0);
	const [favoriteGP, setFavoriteGP] = useState(0);
	const src = GUINEAPATHS[currentGP];

  const favoriteChangeHandler = (event) => {
    setFavoriteGP(parseInt(event.target.value));
  }

  const resetFavoriteHandler = () => {
    setFavoriteGP(0);
  }

  useEffect(() => {
		const intervalId = setInterval(() => {
			setCurrentGP(prevGP => {
				const nextGP = prevGP + 1;
				return nextGP % GUINEAPATHS.length;
			});
		}, 5000)
		return () => clearInterval(intervalId);
	}, []);


	return (
    <>
      <div data-testid="guineaPigsSlideShow" id="guineaPigsSlideShow">
        <h1>Cute Guinea Pigs</h1>
        <img alt="Guinea Pigs Slideshow" src={src} className={currentGP === favoriteGP ? "favorite" : ""}/>
      </div>
		  <div data-testid="guineaPigsForm" id="guineaPigsForm">	
			  <label>Choose Your Favorite Guinea Pig:
          <select value={favoriteGP} onChange={favoriteChangeHandler}>
            <option value="0">Alex</option>
            <option value="1">Izzy</option>
            <option value="2">Brandon</option>
            <option value="3">DJ</option>
          </select>
        </label>
        <button onClick={resetFavoriteHandler}>Reset Favorite</button>
		  </div>
  </>
  );
}

export default GuineaPigs;

#### Create Presentational Component
Now that we’ve created a container component and separated the logic, we can create a presentational (or stateless) component to render our guinea pig slideshow.

The presentational component’s only job is to contain JSX. It should be an exported component and should not render itself because a presentational component will always get rendered by a container component.

For example, say we have components called `Presentational` and `Container`. **Presentational.js** must export the component function (or class, when applicable):
```
function Presentational(/*...props*/) {
  // body of the component                        
}

export default Presentational;
Container.js must import that component:

import { Presentational } from 'Presentational.js';
function Container() {
  // renders Presentational component
}
```

It’s important to understand that although a presentational component doesn’t maintain state, it doesn’t mean it is not reactive. Recall that, like state, a change in props also triggers a potential change in the rendered JSX.



1. Let’s start separating some of the rendering logic from `GuineaPigs` component into a new presentational component called `GuineaPigsSlideShow`.

Near the top left of the code editor, click on the folder icon. Inside **components/**, create a new file named **GuineaPigsSlideShow.js**.

Next, open **components/GuineaPigsSlideShow.js** and create and export a functional component called GuineaPigsSlideShow. Using object destructuring, extract the src and isFavorite props.

Finally, click Run.


2. Open **containers/GuineaPigsContainer.js**.

Locate the `div` with the `id` of `guineaPigsSlideShow`. Copy this `div` and its content.

Now, open **components/GuineaPigsSlideShow.js**. Inside the function, let’s return the JSX you’ve copied.

Finally, the `img`'s `src` attribute value will automatically pick up its new value from the `src` prop. We’ll need to fix the `className` attribute value from `currentGP === favoriteGP? "favorite" : ""` to `isFavorite? "favorite" : ""` since we no longer have direct access to `currentGP` and `favoriteGP`.

In [ ]:
// #+name: components/GuineaPigsSlideShow.js
import React from "react";

function GuineaPigsSlideShow({src, isFavorite}) {

  return (
    <div data-testid="guineaPigsSlideShow" id="guineaPigsSlideShow">
      <h1>Cute Guinea Pigs</h1>
      <img alt="Guinea Pigs Slideshow" src={src} className={isFavorite ? "favorite" : ""}/>
    </div>
  )
}

export default GuineaPigsSlideShow;

#### Parent/Child and Sibling/Sibling Communication
We’ve seen how container components communicate with presentational components by passing their state through props, but how do presentational components communicate changes to the container?

One idea would be to update the props directly like this:
```
function Presentational(props) {
  const buttonClickHandler = () => {
    props.isActive = !props.isActive
  }
  // rest of code
}
```

But this would not be correct because components should never directly update their props. Recall that React functional components should be pure functions and updating prop values directly would violate that principle.

In order for a presentation (stateless) component to communicate changes to a container (stateful) component, the container component must define and provide a way for the presentational component to communicate with it using a change handler function passed as a prop.

For example:
```
function Container() {
  const [isActive, setIsActive] = useState(false);                                
 
  return (
    <>
      <Presentational active={isActive} toggle={setIsActive}/>
      <OtherPresentational active={isActive}/>
    </>
    );                            
  }
 
function Presentational(props) {
  return (
    <h1>Engines are {props.active}</h1>
    <button onClick={() => props.toggle(!props.active)}>Engine Toggle</button>
  );
}
 
function OtherPresentational(props) {
  // render...
}
```

In the example above, `Container` maintains the `isActive` state and passes `setIsActive` to `Presentational` through the `toggle` prop. When `Presentational` needs to communicate a change to the `active` prop, it uses the function passed to it through the `toggle` prop.

Using this pattern also indirectly results in communication between sibling components (components with a common parent), as shown in the example above. When `Presentational` communicates a change through `toggle`, it causes a state update in `Container`, which provides the updated value for `isActive` to both `Presentational` and `OtherPresentational` through the `active` prop.



1. Let’s start separating the form rendering logic from `GuineaPigs` component into a new presentational component called `GuineaPigsForm`.

Near the top left of the code editor, click on the folder icon. Inside **components/**, create a new file named **GuineaPigsForm.js**.

Next, open **components/GuineaPigsForm.js** and create and export a functional component called `GuineaPigsForm`. Using object destructuring, extract `favoriteGP`, `onSelectFavorite`, and `onResetFavorite` props.

Finally, click Run.


2. Open **containers/GuineaPigsContainer.js**.

Locate the `<div>` element with the `id` of `guineaPigsForm`. Copy this `div` and its content.

Now, open **components/GuineaPigsForm.js**. Inside the function, let’s return the JSX you’ve copied.

Finally, replace `resetFavoriteHandler` with `onResetFavorite` and replace `favoriteChangeHandler` with `onSelectFavorite`.

In [ ]:
// #+name: components/GuineaPigsForm.js
import React from "react";

function GuineaPigsForm({favoriteGP, onSelectFavorite, onResetFavorite}) {

  // return JSX from GuineaPigsContainer
  // replace "resetFavoriteHandler" with "onResetFavorite" and 
  // "favoriteChangeHandler" with "onSelectFavorite"
  return (
    <div data-testid="guineaPigsForm" id="guineaPigsForm">	
      <label>Choose Your Favorite Guinea Pig:
        <select value={favoriteGP} onChange={onSelectFavorite}>
          <option value="0">Alex</option>
          <option value="1">Izzy</option>
          <option value="2">Brandon</option>
          <option value="3">DJ</option>
        </select>
      </label>
      <button onClick={onResetFavorite}>Reset Favorite</button>
    </div>
  )
};

export default GuineaPigsForm;

#### Render Presentational Components in Container Component
We’ve learned how to separate logic in a container component and render JSX in a presentational component.

The container component should now render the presentational components instead of rendering JSX. The container component’s state will be passed down as props to the presentation components to keep them reactive.



1. Let’s start by importing the two presentation components we created earlier, **components/GuineaPigsForm.js** and **components/GuineaPigsSlideShow.js**.


2. Next, in the return statement, go ahead and delete the content inside the fragment (`<></>`). Inside the fragment, render the `GuineaPigsSlideShow` with the `src` prop with the value of the `src` state constant and set the value of the `isFavorite` prop to `true` when `currentGP` is equal to `favoriteGP` and `false` otherwise.


3. Inside the fragment, under `GuineaPigsSlideShow`, render the `GuineaPigsForm` with the `favoriteGP` prop with the value of the `favoriteGP` state constant, and set the prop `onSelectFavorite` equal to the `favoriteChangeHandler` function, and the `onResetFavorite` prop equal to `resetFavoriteHandler` function.


4. Let’s change the component’s name in **./containers/GuineaPigsContainer.js** from `GuineaPigs` to `GuineaPigsContainer` and fix the export from `GuineaPigs` to `GuineaPigsContainer`.

Navigate to the **index.js** file and:

- Remove `GuineaPigs` import from **./components/GuineaPigs.js** and import `GuineaPigsContainer` from **./containers/GuineaPigsContainer**.
- Replace `root.render(<GuineaPigs />)` with `root.render(<GuineaPigsContainer />)`

Finally, click run and you should see the application load and “look” the same to the user but for the developer, our application is cleaner and more maintainable.

In [ ]:
// #+name: GuineaPigsContainer.js
import React, { useState, useEffect } from "react";
import { createRoot } from "react-dom/client";
import GuineaPigsForm from "../components/GuineaPigsForm";
import GuineaPigsSlideShow from "../components/GuineaPigsSlideShow";

const GUINEAPATHS = [
  "https://content.codecademy.com/courses/React/react_photo-guineapig-1.jpg",
  "https://content.codecademy.com/courses/React/react_photo-guineapig-2.jpg",
  "https://content.codecademy.com/courses/React/react_photo-guineapig-3.jpg",
  "https://content.codecademy.com/courses/React/react_photo-guineapig-4.jpg",
];

function GuineaPigsContainer() {
  const [currentGP, setCurrentGP] = useState(0);
	const [favoriteGP, setFavoriteGP] = useState(0);
	const src = GUINEAPATHS[currentGP];

  const favoriteChangeHandler = (event) => {
    setFavoriteGP(parseInt(event.target.value));
  }

  const resetFavoriteHandler = () => {
    setFavoriteGP(0);
  }

  useEffect(() => {
		const intervalId = setInterval(() => {
			setCurrentGP(prevGP => {
				const nextGP = prevGP + 1;
				return nextGP % GUINEAPATHS.length;
			});
		}, 5000)
		return () => clearInterval(intervalId);
	}, []);


	return (
    <>
      <GuineaPigsSlideShow src = {src} isFavorite = {currentGP == favoriteGP}/>
      <GuineaPigsForm favoriteGP = {favoriteGP} onSelectFavorite = {favoriteChangeHandler} onResetFavorite = {resetFavoriteHandler} />
  </>
  );
}

export default GuineaPigsContainer;

#### Review
Congrats! You’ve learned your first programming pattern to help organize your React code. You divided a complex React component into a container component and a couple of presentational components.

Here are the steps we took:

- Identified that the original component needed to be refactored: it handled calculations/logic and presentation/rendering.

- Created a container component containing all the stateful logic.

- Created a function that calls the state setter method provided by `useState()`.

- Created and exported presentational components containing only JSX.

- Imported the presentational components into the container component.

- Used the presentational components in the return statement of the container component.

- Passed state and functions used to change state as props to the rendered presentational components.

In this programming pattern, the container component does the work of figuring out what to display using state. The presentational component does the work of actually displaying the state through props. If a component does a significant amount of work in both areas, then that’s a sign that you should use this pattern!

#### Video Player
You just learned your first programming pattern. Let’s put it to use!

For this project, you’ll make three React components work together to create a responsive video player.



1. Click Save, and take a look at your video player in the browser. It looks pretty good! But if you try interacting with it, you’ll find that there’s zero functionality.

Take a look at the `App` component. This component has one property stored as state using `useState()`: a `src` containing the address of a video file. `App`'s job is to pass this `src` down to a stateless component, and to pass *the ability to change the* `src` down to a different stateless component.

Passing `src` is the easier part, so let’s do that first.

Inside of `App`'s return statement, give <Video /> an *attribute*. Make this attribute’s *name* `src`, and the attribute’s *value* equal to `src`.


2. Let’s make <Video /> play its passed-in video file!

Select **Video.js**.

In `Video`'s return statement, give <video /> a `src` attribute. Make `src` equal to the passed-in video file.


3. Alright, the video player works! Now, let’s make the *menu* work as well.

You’ve made `App` pass the `src` down to <Video />. Now `App` needs to pass the ability to *change* the `src` down to <Menu />. If you want to pass the ability to *change* `state`, then first, you need to define a function that calls the state setter function returned by `useState()`.

In **App.js**, define a function called `onSelectVideoHandler()` with a single string parameter called `newVideo`. Use `newVideo` to get a new source from `VIDEOS` and use this as an argument to `setSrc`.


4. If you pass `onSelectVideoHandler()` to <Menu />, then you will give <Menu /> the ability to update <App />‘s state.

In `App`'s return statement, give `Menu` an `onSelectVideo` attribute. Set `onSelectVideo`'s *value* equal to the `onSelectVideoHandler()` function.


5. Alright, now you just have to attach this passed-in function to an event listener!

Select **Menu.js**.

In `Menu`'s return statement, give the <form> element an `onClick` attribute. Set `onClick`'s *value* equal to the passed-in `onSelectVideo` function.

Try selecting a video in the browser.

It doesn’t work! Do you know why?

`onSelectVideo` expects a *string* as an argument. But event handlers are automatically passed *event objects*, not strings.

Let’s fix this next.


6. Before using `onSelectVideo`, we’ll need to *wrap* it in a new function that can take an event object as an argument.

Define a new function in `Menu` named `clickHandler()` with a single parameter called `event`.

Inside the body of `clickHandler()`, declare a new constant named `name`. Set `name` equal to `event.target.value`. This will equal the value of a clicked radio button.

Next, call `onSelectVideo()` with `name` as an argument.


7. Only one more step! You need to use your new wrapper function as an *event handler*.

In `Menu`'s return statement, replace `{props.onSelectVideo}` with `{clickHandler}`.


8. Great job!

`App` passes down `src` to `Video`. `Video` uses this info to display the chosen video.

`App` also passes down *the ability to change* `src` to `Menu`. `Menu` uses this ability to let a user select a new video.

You’ve put together a responsive video player and applied a React programming pattern often used by professional React developers.

In [ ]:
// #+name: App.js
import React, { useState } from 'react';
import { createRoot } from 'react-dom/client';
import Video from './Video';
import Menu from './Menu';

const VIDEOS = {
  fast: 'https://content.codecademy.com/courses/React/react_video-fast.mp4',
  slow: 'https://content.codecademy.com/courses/React/react_video-slow.mp4',
  cute: 'https://content.codecademy.com/courses/React/react_video-cute.mp4',
  eek: 'https://content.codecademy.com/courses/React/react_video-eek.mp4'
};

/*
`App`'s job is to pass the `src` down to a stateless component, and to pass the ability to change the `src` down to a different stateless component.

3. If you want to pass the ability to change `state`, then first, you need to define a function that calls the state setter function returned by `useState()`.

In App.js, define a function called `onSelectVideoHandler()` with a single string parameter called `newVideo`. Use `newVideo` to get a new source from `VIDEOS` and use this as an argument to `setSrc`.
*/
function App() {
	const [src, setSrc] = useState(VIDEOS.fast);

  const onSelectVideoHandler = (newVideo) => {
    setSrc(VIDEOS[newVideo]);
  }

  // 1. Pass `src` down to a stateless component by giving Video an attribute named "src", with value {src}
  // 4. Give Menu an `onSelectVideo` attr, equal to `onSelectVideoHandler()` function
	return (
      <div>
        <h1>Video Player</h1>
        <Menu onSelectVideo={onSelectVideoHandler} />
        <Video src={src} />
      </div>
    );
};

const container = document.getElementById("app");
const root = createRoot(container);
root.render(<App />);

In [ ]:
// #+name: Menu.js
function Menu({ onSelectVideo }) {

    // Wrap `onSelectVideo` function in a new function that an take an event object as an argument
    const clickHandler = (event) => {
      
      // (name = event.target.value) = value of the clicked button
      const name = event.target.value;
  
      // Call `onSelectVideo()` function with `name` as argument
      onSelectVideo(name);
    }
  
    /*
    Attach `onSelectVideo` function to an event listener by setting onClick equal to the passed-in function
    
    6. Need to reference the props object using dot notation first in order to call the `onSelectVideo()` function (i.e. onClick={props.onSelectVideo(name)})
    
    7. Use the new wrapper function (defined above) as an event handler -> replace {props.onSelectVideo(name)} with {clickHandler}
    */
  
      return (
            <form onClick={clickHandler}>
                <input type="radio" name="src" value="fast"/> fast
                <input type="radio" name="src" value="slow" /> slow
                <input type="radio" name="src" value="cute" /> cute
                <input type="radio" name="src" value="eek" /> eek
            </form>
          );
  };
  
  export default Menu;

In [ ]:
// #+name: Video.js
// 2. Video function takes in props as argument to access `src` from App.js
function Video(props) {

    // 2. Make `src` equal to the passed-in video file
    return (
      <div>
        <video src={props.src} controls autostart autoPlay muted/>
      </div>
    );
};
export default Video;

### React Styles
#### Inline Styles and Style Object Variables
There are many different ways to use styles in React. This exercise is focused on two of them: *inline styles* and *style object variables*.

An inline style is a style that’s written as an *attribute*, like this:
```
<h1 style={{ color: 'red' }}>Hello world</h1>
```

Notice that it has double curly braces. The outer curly braces are to note that everything between should be read as JavaScript. The *inner* curly braces create a JavaScript object literal.

However, using inline styles can quickly become messy if you want to apply more than just a few styles. An alternative is to store a style object in a *variable* and then inject that variable as the value of the `style` attribute.

To do this, we can initialize an object with properties and values like so:
```
const darkMode = {
  color: 'white',
  background: 'black';
};
```

Then, the object can be injected to style a component:

```
<h1 style={darkMode}>Hello world</h1>
```


1. Let’s practice styling components using both inline styles and style object variables.

Open **StyleDemo.js**. Starting with inline styling, style the first `<h1>` element inside the `StyleDemo` component.

Give the `<h1>` element an attribute with the name of `style`. The attribute’s *value* should evaluate to this object:

`{ background: 'lightgreen', color: 'darkgreen' }`


2. Next, let’s style using a style object variable.

On a new line, declare a new constant named `myStyle`. Set `myStyle` equal to this object:

```
{
  background: 'lightblue',
  color: 'darkblue'
}
```


3. Style the second `<h1>` element by giving it an attribute with the *name* of style. The attribute’s *value* should be `myStyle`.

In [ ]:
// #+name: StyleDemo.js
import React from 'react';

function StyleDemo() {
  const myStyle = 
  {
    background: 'lightblue',
    color: 'darkblue'
  };

  return (
    <>
      <h1 style={{background: 'lightgreen', color: 'darkgreen'}}>Style This With Inline Styling</h1>
      <h1 style={myStyle}>Style This With Style Object Variable</h1>
    </>
  );
}

export default StyleDemo

#### Style Syntax
There are a few things to keep in mind when styling components with JSX.

Just like how we reference CSS properties in the `style` object of the DOM in JavaScript, we write CSS property names using *camelCase* in React:
```
const styles = {
  marginTop: '20px',
  backgroundColor: 'green'
};
```

This syntax comes from a small rule. A hyphen is a reserved operator in JavaScript. If we use `background-color`, the hyphen is then interpreted as a minus sign. Thus, we want to be consistent with the property names in the DOM `style` JavaScript object and use camel case.

In regular JavaScript, style *values* are almost always strings. Even if a style value is numeric, you usually have to write it as a string so that you can specify a unit. For example, you’d write `'450px'` or `'20%'`.

If you write a style value as a *number*, then the unit `'px'` is assumed. For example, if you want a font size of 30px, you can write:

`{ fontSize: 30 }`
If you want to use units other than `'px'`, you can use a string:

`{ fontSize: "2em" }`
Specifying the `'px'` unit in a string will still work, although it’s redundant.

A few specific styles will *not* automatically fill in the `px` for you. These are styles where you aren’t likely to use `'px'` anyway, so you don’t really have to worry about them.



1. Open **StyleDemo.js**.

Find your style object variable, and give it two more properties: a `marginTop` of `'100px'`, and a fontSize of `'50px'`.


2. Next, test out React’s value syntax by changing any property values that end in `'px'` from strings into numbers.

In [ ]:
// #+name: StyleDemo.js
import React from 'react';
const myStyle = {
  background: 'lightblue',
  color: 'darkblue',
  marginTop: 100,
  fontSize: 50
};

function StyleDemo() {
  return <button style={myStyle}>button</button>
}

export default StyleDemo

#### Multiple Stylesheets
While inline styles and style object variables are valid methods of styling in React, it can become organizationally difficult to keep track of the styles being applied as your application grows.

One way to make styles modular, organized, and reusable is to create separate stylesheets for each component.

We can import a stylesheet by using the `import` keyword:

`import './App.css'`

However, if we have multiple stylesheets with the same class names, the names can collide and create style conflicts.

One way to prevent this is to use CSS modules. By importing it as a module, the styles will only be available for the component that imported the style. This is done automatically by creating unique class names for each module. This frees us from having to keep track of the class names we’ve used across stylesheets.

To use CSS modules, we begin by naming our stylesheet in this format, where `fileName` should be replaced with the name of the component you’re styling:

`fileName.module.css`

This indicates that the file should be processed as a CSS module.

Then, it must be imported into the file containing our component.

`import styles from './fileName.module.css'`

From this import, we can see that the `styles` object now holds the class selectors of `fileName.module.css`. To access the selectors, we use the dot notation like so:

`<div className={styles.divStyle}></div>`

Note that we apply styles using the `className` attribute rather than `class`. `class` is a reserved JavaScript keyword, so React uses `className` to avoid conflicts.

Although React does not have an opinion on how styles should be defined, this is the preferred method for styling in React, as it maintains the compositional philosophy of React.



1. Let’s style Codey’s developer portfolio with CSS modules.

Take a look at the existing code. The webpage consists of two components, `Home` and `AttentionGrabber`.

Currently, they are unstyled. But, we have CSS modules **Home.module.css** and **AttentionGrabber.module.css** located inside the **styles** folder.

Open up **AttentionGrabber.js** and import the styles from **AttentionGrabber.module.css**. Store them in a `styles` object.


2. Apply the style to the <h1> element in the AttentionGrabber component.

Start by giving the <h1> tag a className attribute, then set the value to styles.h1.


3. Great! Next, style the Home component.

Open up Home.js and import the styles from Home.module.css. Store them into a styles object.


4. Style the <div> element by giving it the className attribute and the value styles.div.

Click Run, and observe the styles being applied!

In [ ]:
// #+name: AttentionGrabber.js
import React from 'react';

// new import can be any name, no need to definitely be "styles"
import styles from './styles/AttentionGrabber.module.css';


function AttentionGrabber() {
  return (
    <>
      <h1 className={styles.h1}>Hi! Welcome to my portfolio.</h1>
      <p>I'm a developer who likes designing user-friendly websites.</p>
    </>
  )
}

export default AttentionGrabber

In [ ]:
// #+name: Home.js
import React from 'react';

import  AttentionGrabber from './AttentionGrabber';

// new import can be any name, no need to be "styles"
import styles from './styles/Home.module.css';

function Home() {
  return (
    <div className={styles.div}>
      <AttentionGrabber />
      <footer>Codey Cademy: Full-Stack Developer</footer>
    </div>
  );
}

export default Home

#### Review
Well done! You’ve reached the end of this lesson on styling in React apps.

Before you go, here’s a recap:

- React components can be styled in a number of different ways: inline styling, object variable styling, stylesheets, and CSS modules.

- Inline styling can be used to apply styles to a single element. Inline styling can be done by giving the element an attribute named `style` whose value is an object literal surrounded in curly braces.

<h1 style={{color: "red"}}> Hello, World! </h1>

- An object variable can also be used to apply a style to a single element. The syntax is similar to inline styling, but rather than passing an object literal, the name of the variable is passed instead.

const myStyle = { color: "red" }
<h1 style={myStyle}> Hello, World! </h1>

- Style names in React must be in camelCase. For example, `background-color` becomes `backgroundColor`.

- In React, a number style value is automatically interpreted with `px`.

- Styles can be separated and stored into CSS module files. The styles can be imported and used by applying `className` attributes to the relevant elements.

#### Styling Rock, Paper, Scissors
In this project, you will get the chance to practice styling React applications using different techniques.

You will be styling a game of Rock, Paper, Scissors. However, rather than sticking to one technique, you will be asked to modify its appearance using inline styling syntax, object variable syntax, and, finally, CSS modules!

Throughout the project, you’ll be able to explore the advantages and disadvantages of each approach and practice React naming conventions for style properties.

By the end of this practice project, you will have improved your skills in styling React applications using various techniques and gained a deeper understanding of the different approaches. You will be able to apply your learnings to future React projects and improve your coding skills!

Let’s roll up our sleeves and start!



**Inline Styling**
1. Take a look at **Game.js**. The game is already functional, but it needs some styling.

Start by practicing your understanding of inline styling.

Find the <h1> element responsible for the game’s title, “Rock Paper Scissors”.

Apply an inline style to change the font size property to `48`, and set the margin top property to `0`.


**Object Variable Styling**
2. Next, let’s define a few object variables to use for styling.

Above the `Game` component, create an object variable named `choiceStyles`. The object should contain the following properties:

  - Display with the value of `flex`.
  - Align items with the value of `center`.
  - Margin bottom with a value of `40` pixels.


3. Under the `choiceStyles`, define another object variable named `emojiStyles` with the following properties:

Font size with the value `64` pixels.
Margin right with the value of `20` pixels.


4. Under the `emojiStyles`, define another object variable named `nameStyles` with the following properties:

  - Margin of 0 pixels.
  - Font size of 24 pixels.
  - Color with the value of #ffff.


5. Last, under the `nameStyles`, define another object variable named `resultStyle` with the following properties:

Margin top of `40` pixels.
Font size of `48` pixels.
Color with the value of `#ffff`.


6. Great job! Our object variable styles are now defined. It’s time to apply it.

Apply the styles in `choiceStyles` to the <div> surrounding:
```
<span>{playerChoice.emoji}</span>
<p>You chose {playerChoice.name}</p>
```

and the <div> container surrounding:
```
<span>{codeyChoice.emoji}</span>
<p>The computer chose {codeyChoice.name}</p>
```

7. Apply the styles in `emojiStyles` to:
```
<span>{playerChoice.emoji}</span>
```

as well as:
```
<span>{codeyChoice.emoji}</span>
```

8. Apply the styles in `nameStyles` to:

<p>You chose {playerChoice.name}</p>


as well as:

<p>Codey chose {codeyChoice.name}</p>


9. Last, apply `resultStyle` to:

<h2>{result}</h2>


10. Click Save to run your code. Your page should render with some styles applied. It doesn’t look great yet, because we’re not done with the styling—but if you see the text and emojis in a bigger size, you’re on the right track!

Let’s keep going.


**CSS Module Styling**
11. Let’s proceed to styling with CSS modules now.

Open up **Game.module.css**. Take a look inside. There are already styles defined in there for you. All we have to do is apply it!

Go back to **Game.js** and apply the styles for the `.container` selector to the very first <div> after in the return statement:
```
return (
  <div>
    <h1 style={{ fontSize: 48, marginTop: 0 }}>Rock Paper Scissors</h1>
    ...
  </div>
);
```

Remember to import the CSS module into an object named `styles`!


12. Apply the styles for the `.choices` selector to the <div> containing the rock, paper, scissors buttons.

You can find this <div> under the <h1> title and above the logic for mapping over the `CHOICES` object array.


13. Next, apply the `.results` selector to the <div> containing the game’s results.

You can find this <div> between `playerChoice && codeyChoice && (` and <div style={choiceStyles}>.


14. Lastly, apply the styles for the `.button` selector to the `Play Again` button and each choice button.


15. Press Save to run and see how the Rock Paper Scissors game has been fleshed out and styled in three different ways!

In [ ]:
// #+name: Game.module.css
.container {
    display: flex;
    flex-direction: column;
    align-items: center;
    font-family: "Courier New", Courier, monospace;
    background-color: #000000;
    color: #ffffff;
    padding: 40px;
  }
  
  .choices {
    display: flex;
    justify-content: center;
    margin: 40px 0;
  }
  
  .button {
    font-size: 48px;
    background-color: #333333;
    color: #ffffff;
    border: 4px solid #ffffff;
    border-radius: 8px;
    padding: 20px 40px;
    margin: 0 20px;
    cursor: pointer;
    transition: all 0.2s ease-in-out;
  }
  
  .results {
    display: flex;
    flex-direction: column;
    align-items: center;
    margin-top: 40px;
  }  

In [ ]:
// #+name: Game.js
import React, { useState } from "react";

// import the defined styles
import styles from './Game.module.css';

const CHOICES = [
  { name: "rock", emoji: "✊" },
  { name: "paper", emoji: "✋" },
  { name: "scissors", emoji: "✌️" },
];

// object variables to use for styling
const choiceStyles = 
{
  display: 'flex',
  alignItems: 'center',
  marginBottom: '40px'
};

const emojiStyles = 
{
  fontSize: '64px',
  marginRight: '20px'
};

const nameStyles = 
{
  margin: '0px',
  fontSize: '24px',
  color: '#ffff'
};

const resultStyle = 
{
  marginTop: '40px',
  fontSize: '48px',
  color: '#ffff'
};


function Game() {
  const [playerChoice, setPlayerChoice] = useState(null);
  const [codeyChoice, setCodeyChoice] = useState(null);
  const [result, setResult] = useState(null);

  function handlePlayerChoice(choice) {
    const codeyChoice = CHOICES[Math.floor(Math.random() * CHOICES.length)];
    setPlayerChoice(choice);
    setCodeyChoice(codeyChoice);
    if (choice.name === codeyChoice.name) {
      setResult("Tie!");
    } else if (
      (choice.name === "rock" && codeyChoice.name === "scissors") ||
      (choice.name === "paper" && codeyChoice.name === "rock") ||
      (choice.name === "scissors" && codeyChoice.name === "paper")
    ) {
      setResult("You win!");
    } else {
      setResult("You lose!");
    }
  }

  function resetGame() {
    setPlayerChoice(null);
    setCodeyChoice(null);
    setResult(null);
  }

  // in-line styling on h1
  // apply object variable styles based on requirements
  // apply style for `.container` selector using className
  return (
    <div className={styles.container}>
      <h1 style={{ fontSize: 48, marginTop: 0 }}>Rock Paper Scissors</h1>
      <div className={styles.choices}>
        {CHOICES.map((choice) => (
          <button
            key={choice.name}
            onClick={() => handlePlayerChoice(choice)}
            aria-label={choice.name}
          >
            {choice.emoji}
          </button>
        ))}
      </div>
      {playerChoice && codeyChoice && (
        <div className={styles.results}>
          <div style={choiceStyles}>
            <span style={emojiStyles}>{playerChoice.emoji}</span>
            <p style={nameStyles}>You chose {playerChoice.name}</p>
          </div>
          <div style={choiceStyles}>
            <span style={emojiStyles}>{codeyChoice.emoji}</span>
            <p style={nameStyles}>The computer chose {codeyChoice.name}</p>
          </div>
          <h2 style={resultStyle}>{result}</h2>
          <button className={styles.button} onClick={resetGame}>Play again</button>
        </div>
      )}
    </div>
  );
}

export default Game;

### React Forms
#### Input onChange
Let’s talk about how forms are handled in React.

In a regular HTML form, the state of the form is typically managed by the browser. It doesn’t update the server until the user hits submit.

Things work a bit differently in React. In a React form, the state of the form can be managed by the component, and updates are triggered by the `onChange` event. When the user interacts with an input, such as entering or deleting any characters, the `onChange` event fires and updates the component state.

This allows the component to immediately reflect any changes made by the user and update the view accordingly.

Let’s dive into how it works.



1. Take a look at the existing code. This is an input form for a newsletter! The form currently only asks for the user’s email address. Under it is some text stating “Current User Input”. We’ll use the space here to see the data the user is entering.

Now, open **Input.js**.

View the return statement and try typing into the input field on the browser.

Then, press Run!


2. Not much happened! Current User Input in the browser still shows nothing despite there being input! Don’t worry; once `Input` has been set up correctly, then you will be able to see the data received from the <input> element live as you type.


3. To change our code so that it responds to any changes in the input field, start by listening for a “change” event on the <input> tag.

Give <input> an `onChange` attribute. Set `onChange`‘s *value* equal to `{handleUserInput}`.

Don’t worry about the fact that there is no `handleUserInput()` function yet. You’ll write one!

In [ ]:
// #+name: Input.js
import React, { useState } from 'react';
import styles from './Input.module.css';

function Input() {
  return (
    <>
      <div className={styles.emailContainer}>
        <h2>Let's stay in touch.</h2>
        <p>Sign up for our newsletter to stay up-to-date on the latest products, receive exclusive discounts, and connect with other programmers who share your passion for all things tech.</p>
        <form>
          <label for="email">Email: </label>
          <input id="email" type="text" onChange={handleUserInput}/>
        </form>
      </div>
      <div className={styles.inputDisplay}>
        <h2>Current User Input: </h2>
        <h4></h4>
      </div>
    </>
  );
}

export default Input;

#### Write an Input Event Handler
Let’s define an event handler function that gets called whenever a user enters or deletes any character inside the <input> element.

The event handler function will listen for `change` events. You can see an example of an event handler listening for change events in **Example.js**.



1. Select **Input.js**.

Before `Input`‘s return statement, write a new function called `handleUserInput()`.

Give this function one parameter named `e`.


2. Inside the `handleUserInput()` function’s body, call `setUserInput`. Set the state’s `userInput` property equal to `e.target.value`.

`e.target.value` will equal the text in the <input> field. You are setting userInput equal to whatever text is currently in <input>.

In [ ]:
// #+name: Example.js
import React, { useState } from "react";

function Example() {
  const [userInput, setUserInput] = useState("");
  function handleChange(e) {
    setUserInput(e.target.value);
  }
  return <input onChange={handleChange} type="text" />;
}

In [ ]:
// #+name: Input.js
import React, { useState } from 'react';
import styles from './Input.module.css';

function Input() {
    
  // new function to handle user input
  function handleUserInput(e){
    setUserInput(e.target.value);
  }
  
  return (
    <>
      <div className={styles.emailContainer}>
        <h2>Let's stay in touch.</h2>
        <p>Sign up for our newsletter to stay up-to-date on the latest products, receive exclusive discounts, and connect with other programmers who share your passion for all things tech.</p>
        <form>
          <label for="email">Email: </label>
          <input id="email" type="text" onChange={handleUserInput}/>
        </form>
      </div>
      <div className={styles.inputDisplay}>
        <h2>Current User Input: </h2>
        <h4></h4>
      </div>
    </>
    );
}

export default Input

#### Set the Input's Initial State
Good! Any time that someone types or deletes in the <input> field, the `handleUserInput()` function will update `userInput` with the <input>‘s text.

Since we’re using `setUserInput`, that means that `userInput` needs an initial state! We’ll need to use the State hook. What should the state’s initial *value* be?

Well, `userInput` will be displayed in the <input> element. What should the *initial* text in the <input> be when a user first visits the page?

The initial text should be an empty string! Otherwise, it would look like someone had already typed something.


1. Use a State Hook to define the state variable userInput and the setUserInput state setter function.


2. Initialize the value of userInput to an empty string.

In [ ]:
// #+name: Input.js
import React, { useState } from "react";
import styles from "./Input.module.css";

function Input() {

  // use State Hook to define state variable and state setter function
  const [userInput, setUserInput] = useState('');
  
  function handleUserInput(e) {
    setUserInput(e.target.value);
  }
  return (
    <>
      <div className={styles.emailContainer}>
        <h2>Let's stay in touch.</h2>
        <p>
          Sign up for our newsletter to stay up-to-date on the latest products,
          receive exclusive discounts, and connect with other programmers who
          share your passion for all things tech.
        </p>
        <form>
          <label for="email">Email: </label>
          <input id="email" type="text" onChange={handleUserInput} />
        </form>
      </div>
      <div className={styles.inputDisplay}>
        <h2>Current User Input: </h2>
        <h4></h4>
      </div>
    </>
  );
}

export default Input;

#### Update an Input's Value
When a user types in the <input> field, then that will trigger a *change* event, which will call `handleUserInput()`. That’s good!

`handleUserInput()` will set `userInput` equal to whatever text is currently in the <input> field. That’s also good!

There’s only one problem: you can set `userInput` to whatever you want, but <input>‘s `value` prop will not update.

In React, the `value` prop of an input element is used to control the value of the input and keep it in sync with the component’s state. Without setting the `value` prop, changes made to the input would not be reflected in the component’s state, leading to inconsistencies and potential bugs down the line.

To ensure that the input’s value matches the component state, we can set the `value` prop and use the `onChange` event to update the state with the new value. When the state is updated, the component re-renders, and the `value` prop is set to the new value from the component’s state.

This makes the component state the “source of truth” for the input’s value, ensuring that the form data is consistent and can be easily managed and submitted.

For example, if we had a login form with an email and password input, we would set the value prop for both inputs and update the component’s state whenever the user types in a new email or password. This way, the form data is always up to date with the user’s input.



1. Inside of `Input`'s return statement, give the <input> tag a `value` attribute with the value of `{userInput}`.


2. That should do it! An idiomatically correct React form!

This example doesn’t have a server per se, but any time that a user updates <input>, the new text is immediately stored in `Input`'s `state`. We could easily connect that `state` with a server. What matters is that the text is sent somewhere to be stored on every character change.

Inside of the <h4></h4> tags, write:

`{userInput}`

Click Run. Try typing into the <input> field in the browser.

In [ ]:
// #+name: Input.js
import React, { useState } from "react";
import styles from "./Input.module.css";

function Input() {
  const [userInput, setUserInput] = useState('');
  
  function handleUserInput(e) {
    setUserInput(e.target.value);
  }
  
  return (
    <>
      <div className={styles.emailContainer}>
        <h2>Let's stay in touch.</h2>
        <p>
          Sign up for our newsletter to stay up-to-date on the latest products,
          receive exclusive discounts, and connect with other programmers who
          share your passion for all things tech.
        </p>
        <form>
          <label for="email">Email: </label>
          <input id="email" type="text" onChange={handleUserInput} value={userInput}/>
        </form>
      </div>
      <div className={styles.inputDisplay}>
        <h2>Current User Input: </h2>
        <h4>{userInput}</h4>
      </div>
    </>
  );
}

export default Input;

#### Controlled vs Uncontrolled
There are two terms that will probably come up when you talk about React forms: **controlled component** and **uncontrolled component**.

An *uncontrolled component* is a component that maintains its own internal state. A *controlled component* is a component that does not maintain any internal state. Since a controlled component has no state, it must be *controlled* by someone else.

Think of a typical <input type='text' /> element. It appears on the screen as a text box. If you need to know what text is currently in the box, then you can ask the <input> element, possibly with some code like this:

```
let input = document.querySelector('input[type="text"]');
 
let typedText = input.value; // input.value will be equal to whatever text is currently in the text box.
```

The important thing here is that the <input> element *keeps track of* its own text. You can access what its text is at any time.

The fact that <input> keeps track of information makes it an *uncontrolled component*. It maintains its own internal state by remembering data about itself.

A controlled component, on the other hand, has no memory. If you ask it for information about itself, then it will have to get that information through `props`. Most React components are *controlled*.

In React, when you give an <input> element a `value` attribute, then something strange happens: the <input> element becomes *controlled*. It stops using its internal storage. This is a more “React” way of doing things.

Recall that in our exercises, the page updated every time we typed into the input. React controlled the input’s value with the state. We’ve been demonstrating the idea of a controlled component all along!

You can find more information about controlled and uncontrolled components in the React documentation.

#### Review
Great work! You just wrote your first React form.

Here’s a review:

- The state of a React form is managed by the component, and updates are triggered by the onChange event.

- The onChange event uses an event handler to capture changes and determine what actions to take.

- A React form uses the State hook to store the value of the input field in the component’s state. The state can then be updated with the state setter.

- React components can be controlled or uncontrolled. Most React forms are controlled, as they control the input’s value with the state.

That marks the end of this lesson! The skills you’ve learned with React forms will be invaluable as you continue to build out more React apps!

#### Saucy Tango Food Order Form
In this practice project, you will create a food order form for a restaurant.

The local mom-and-pop shop, Saucy Tango, has received a lot of love from the community recently. To keep up with demand, they’ve decided to open their restaurant for online orders. They’ve entrusted this task to you, a budding React developer.

Your goal is to create a functional form that allows customers to place their orders easily. Once the customer submits the form, the form will alert them about their order details. You will be provided with a menu and your task is to create a form that can take in customer details such as name, phone number, address, and order.

To complete this project, you will use your understanding of controlled React forms. You’ll be using the State Hook to manage the state of the form, and employ event handlers to update the state of the form as the customer fills in their details.

Let’s dive in and get started!



**Setting Up the Form**
1. Open up **FoodOrderForm.js**. We’ll start laying down the foundation of our form.

Inside the `FoodOrderForm` component’s return statement, add a <form> element.


2. Inside the <form></form> tags, create an <input> field each for `name`, `phone`, `address`, and `order`.

For each <input>, add an `id` attribute set to the information it is responsible for, such as `"name"`, `"phone"`, `"address"`, and `"order"`.


3. Attach a <label> element to each <input> element.

Each label should have the attribute `htmlFor` with a string value of the corresponding <input> element’s `id`. This will associate the <label> with its respective <input> element and improve accessibility for users.


4. Great, the foundation is taking shape!

Let’s now add a button at the bottom of our form. You can set the text of the button to `"Submit Order"` and include a `type` attribute with a value of `"submit"`. This will let the browser know that this button is intended to submit the form.


**Controlling the Form**
5. Currently, our form is *uncontrolled*. It does not use React to control its state and update the values of the input field.

We need to change that and hand the control to React. Take a moment to consider what form data you should be storing and how a State Hook can help you with that.

Since our form keeps track of information like `name`, `phone`, `address`, and `order`, let’s create a state for each piece of information.

Inside the `FoodOrderForm` component and above the return statement, use the State Hook to define a state variable for each piece of information, along with their state setters.

Initialize all state variables to an empty string.


6. Now that we have the state variables set up, we can make some modifications to our <input> elements.

For each <input> element, add a `value` attribute and set the value to the corresponding state variable.


7. Wonderful. React now has control over the form!

This form still needs to update the input field and state variables when the user types. To do this, you need to use the `onChange` event listener to listen for changes.

Update each <input> element by giving it an `onChange` event handler.

Start with the `name` input. Inside the `onChange` event handler, give it this callback function.
```
onChange={(e) => setName(e.target.value)}
```

This calls the corresponding state setter function `setName` and passes in the new value of the input field retrieved from the event object.

This will update the state of the corresponding state variable with the new value entered by the user.


8. Repeat the same for the remaining <input> elements, just as you did with the `name` input field.

Give each <input> element an `onChange` event handler and pass in a callback function that updates the corresponding state variable of each input based on the values of `e.target.value`.


**Form Submission**
9. Congrats! Your form now updates the state variable live as the user types, and the page re-renders to reflect that.

You need to now handle the submission of the form.

Inside the `FoodComponentForm` component, define a `handleSubmit()` function to manage the submitted form data. The function should receive an event object, `e`, in its parameter.

In the function body, prevent the form from taking its default action with `e.preventDefault()`. Rather than using the default action, we want to have it alert the user instead.


10. While still in the function body of `handleSubmit()`, alert the user using the JavaScript function `alert()`.

The alert should output the user’s `order` data in this format.
```
Order Successful!
 
Your order was <user's orders>.
 
Please show your confirmation number for pickup.
```

11. Pass the `handleSubmit()` function to the form by giving the form tag an `onSubmit` event listener with the value `handleSubmit`.


12. Wonderful! The form is working quite well already.

Let’s clean up a little and add some necessary information before submitting it to Saucy Tango for use.

Complete each <input> element by giving each a `name` attribute and a `type` attribute, specifying the appropriate name and type for each input.

In [ ]:
// #+name: FoodOrderForm.js
import React, { useState } from "react";

function FoodOrderForm() {
  // 4. Use State Hook to define a state variable for each piece of info. Initialize all to empty string
  const [name, setName] = useState("");
  const [phone, setPhone] = useState("");
  const [address, setAddress] = useState("");
  const [order, setOrder] = useState("");


  // 9. Manage the submitted form data
  const handleSubmit = (e) => {
    // preventDefault as we want to alert user instead
    e.preventDefault();
    
    const alertString = `Order Successful!\nYour order was \nName: ${name}\nPhone: ${phone}\nAddress: ${address}\nOrder: ${order}\nPlease show your confirmation number for pickup.`
    
    alert(alertString);

    // Clear inputs after alert
    setName('');
    setPhone('');
    setAddress('');
    setOrder('');
  }


  /* 
  2. return a form with inputs for each field
  3. attach a label with (htmlFor) attribute for association to each input. `for` is a reserved JS keyword so use `htmlFor`
  6. Add a `value` attribute for each input, set to the corresponding state variable
  
  7. `onChange` event listener to update input field and state variables when user types.

  onChange={(e) => setName(e.target.value)}
  
  Calls the corresponding state setter function setName and passes in the new value of the input field retrieved from the event object.

This will update the state of the corresponding state variable with the new value entered by the user.

  11. Pass `handleSubmit()` function to the form
  */
  return (
    <form onSubmit={handleSubmit}>
      <label htmlFor="name">Name: </label>
      <input 
        id="name" 
        value={name} 
        onChange={(e) => setName(e.target.value)}
        name="name"
        type="text" 
      />

      <label htmlFor="phone">Phone: </label>
      <input 
        id="phone" 
        value={phone} 
        onChange={(e) => setPhone(e.target.value)}
        name="phone"
        type="text"  
      />

      <label htmlFor="address">Address: </label>
      <input 
        id="address" 
        value={address} 
        onChange={(e) => setAddress(e.target.value)}
        name="address"
        type="text"  
      />

      <label htmlFor="order">Order: </label>
      <input 
        id="order" 
        value={order} 
        onChange={(e) => setOrder(e.target.value)}
        name="order"
        type="text"  
      />
      
      <button type="submit">Submit Order</button>
    </form>
  )
}

export default FoodOrderForm;

In [ ]:
// #+name: Header.js
import React from "react";
import styles from './Header.module.css'


const Header = () => {
  return (
    <header className={styles.header}>
      <h1>Saucy Tango</h1>
      <p>Indulge in Saucy Tango's rich flavors and bold spices entice your taste buds to dance the night away.</p>
    </header>
  );
};

export default Header;

In [ ]:
// #+name: FoodItem.js
import React from "react";
import styles from "./FoodItem.module.css";

const FoodItem = ({ name, price, imgSrc }) => {
  return (
    <div className={styles.card}>
      <div className={styles.imageWrapper}>
        <img src={imgSrc} alt="" className={styles.image} />
      </div>
      <div className={styles.detailsWrapper}>
        <h3>{name}</h3>
        <p className={styles.details}>{`Price: ${`${price}`}`}</p>
      </div>
    </div>
  );
};

export default FoodItem;

In [ ]:
// #+name: App.js
import React from 'react';
import FoodOrderForm from './FoodOrderForm';
import FoodItem from "./FoodItem";
import styles from "./App.module.css";
import Header from './Header.js';

function App() {
  return (
    <div className={styles.container}>
      <Header />
      <FoodItem name="Shakshuka" price={5.99} imgSrc="https://images.unsplash.com/photo-1590412200988-a436970781fa?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=870&q=80"/>
      <FoodItem name="Spaghetti Carbonara" price={6.99} imgSrc="https://images.unsplash.com/photo-1608756687911-aa1599ab3bd9?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=774&q=80" />
      <FoodItem name="Margherita Pizza" price={2.99} imgSrc="https://images.unsplash.com/photo-1589187151053-5ec8818e661b?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=774&q=80"/>
      <FoodOrderForm />
    </div>
  )
}

export default App

In [ ]:
// #+name: Index.js
import React from 'react';
import ReactDOM from 'react-dom/client';

import App from './App';

ReactDOM.createRoot(
  document.getElementById('app')
).render(<App />);